# GPT-OSS 20B Reinforcement Learning on Tau-Bench Tasks

Train GPT-OSS 20B to perform customer service tool calling using tau-bench style tasks with Reinforcement Learning (GRPO).

**Key Features:**
- Uses tau-bench dataset.jsonl generated by task_generator.py
- Implements reward function based on action correctness and execution success
- Optimized for DGX systems with Unsloth (70% VRAM reduction, 2-6x faster)

## Installation

Install Unsloth and dependencies for RL training on GPT-OSS 20B.

In [1]:
%%capture
import os, importlib.util
%pip install --upgrade -qqq uv

if importlib.util.find_spec("torch") is None or "COLAB_" in "".join(os.environ.keys()):
    try: import numpy; get_numpy = f"numpy=={numpy.__version__}"
    except: get_numpy = "numpy"
    %uv pip install -qqq \
        "torch>=2.8.0" "triton>=3.4.0" {get_numpy} torchvision bitsandbytes "transformers==4.56.2" \
        "unsloth_zoo[base] @ git+https://github.com/unslothai/unsloth-zoo" \
        "unsloth[base] @ git+https://github.com/unslothai/unsloth" \
        git+https://github.com/triton-lang/triton.git@05b2c186c1b6c9a08375389d5efe9cb4c401c075#subdirectory=python/triton_kernels
elif importlib.util.find_spec("unsloth") is None:
    %uv pip install -qqq unsloth

%uv pip install --upgrade --no-deps transformers==4.56.2 tokenizers trl==0.22.2 unsloth unsloth_zoo

## Load Model with Unsloth

Load GPT-OSS 20B with optimizations:
- `max_seq_length = 2048`: Context length for longer tool calling sequences
- `lora_rank = 16`: LoRA rank for efficient RL (higher = smarter but slower)
- `load_in_4bit = True`: 4-bit quantization to save 75% VRAM
- `offload_embedding = True`: Reduce VRAM by 1GB

In [2]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048  # Longer context for tool calling
lora_rank = 16         # Larger rank for better RL performance

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gpt-oss-20b",  # Use gpt-oss-20b-BF16 on H100/B200
    max_seq_length = max_seq_length,
    load_in_4bit = True,       # False for 16-bit on H100s
    offload_embedding = True,  # Reduces VRAM by 1GB
)

print(f"✓ Model loaded successfully!")

[unsloth.import_fixes|INFO]Unsloth: Patching protobuf.MessageFactory.GetPrototype
[unsloth.import_fixes|INFO]Unsloth: torch==2.10.0a0+b558c986e8.nv25.11 and torchvision==0.25.0a0+7a13ad0f are compatible.


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/usr/local/lib/python3.12/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[unsloth_zoo.log|INFO]Unsloth: set VIDEO_TOTAL_PIXELS: 90316800
[unsloth_zoo.log|INFO]Unsloth: Patched Gemma3Processor.__call__.
[unsloth_zoo.log|INFO]Unsloth: Patched Gemma3RMSNorm.forward.
[unsloth_zoo.log|INFO]Unsloth: Patched Gemma3Attention.forward.
[unsloth_zoo.log|INFO]Unsloth: Patched AutoHfQuantizer.merge_quantization_configs.
[unsloth_zoo.log|ERROR]Unsloth: Skipped CsmDepthDecoderForCausalLM.forward
Reason: New function removed output_attentions and output_hidden_states
[unsloth_zoo.log|INFO]Unsloth: Patched CsmDepthDecoderForCausalLM.forward.
[unsloth_zoo.log|ERROR]Unsloth: Skipped CsmForConditionalGeneration.forward
Reason: New function removed output_attentions and output_hidden_states
[unsloth_zoo.log|INFO]Unsloth: Patched CsmForCo

🦥 Unsloth Zoo will now patch everything to make training faster!


[unsloth_zoo.log|INFO]Unsloth: Patching trl openenv with function: openenv_vllm_reload_weights


Unsloth: Could not import trl.trainer.nash_md_trainer: Failed to import trl.trainer.nash_md_trainer because of the following error (look up to see its traceback):
cannot import name 'amp' from 'apex' (/usr/local/lib/python3.12/dist-packages/apex/__init__.py)
Unsloth: Could not import trl.trainer.online_dpo_trainer: Failed to import trl.trainer.online_dpo_trainer because of the following error (look up to see its traceback):
cannot import name 'amp' from 'apex' (/usr/local/lib/python3.12/dist-packages/apex/__init__.py)
Unsloth: Could not import trl.trainer.xpo_trainer: Failed to import trl.trainer.xpo_trainer because of the following error (look up to see its traceback):
cannot import name 'amp' from 'apex' (/usr/local/lib/python3.12/dist-packages/apex/__init__.py)
Unsloth: Bitsandbytes >= 0.46.0 supports torch.compile - enabling.
Unsloth: Patched cross entropy losses.
No module named 'causal_conv1d' No module named 'causal_conv1d'
Unsloth: Failed compiling causal_conv1d
Unsloth: Failed

[unsloth_zoo.log|INFO]Unsloth: Patched Gemma3Processor.__call__.
[unsloth_zoo.log|INFO]Unsloth: Patched Gemma3RMSNorm.forward.
[unsloth_zoo.log|INFO]Unsloth: Patched Gemma3Attention.forward.
[unsloth_zoo.log|ERROR]Unsloth: Skipped CsmDepthDecoderForCausalLM.forward
Reason: New function removed output_attentions and output_hidden_states
[unsloth_zoo.log|INFO]Unsloth: Patched CsmDepthDecoderForCausalLM.forward.
[unsloth_zoo.log|ERROR]Unsloth: Skipped CsmForConditionalGeneration.forward
Reason: New function removed output_attentions and output_hidden_states
[unsloth_zoo.log|INFO]Unsloth: Patched CsmForConditionalGeneration.forward.
[unsloth_zoo.log|INFO]Unsloth: Patched CsmForConditionalGeneration._merge_input_ids_with_input_values.
[unsloth_zoo.log|INFO]Unsloth: Patched Gemma3nTextAltUp.predict.
[unsloth_zoo.log|INFO]Unsloth: Patched Gemma3nTextAltUp.correct.
[unsloth_zoo.log|INFO]Unsloth: Patched Gemma3nTextAltUp.scale_corrected_output.
[unsloth_zoo.log|ERROR]Unsloth: Skipped Gemma3nMod

Unsloth: Disabling compile for GptOssExperts since it's marked for disabling.
Unsloth: Disabling compile for GptOssMLP since it's marked for disabling.
Unsloth: Will not compile GptOssAttention since it looks like it calls attention modules!
Unsloth: Compiled module GptOssRMSNorm.
Unsloth: Compiled module GptOssTopKRouter.
Unsloth: Compiled module GptOssRotaryEmbedding.
(2) Unsloth skipping patching fast linear cross entropy for GptOssForCausalLM
[2/3 pattern] Successfully patched fast linear cross entropy for GptOssForCausalLM
Unsloth: Fast fused linear cross entropy patch for GptOssForCausalLM.
Unsloth: Fixed up function apply_rotary_pos_emb.
Unsloth: Compiled function repeat_kv.
Unsloth: Compiled function _apply_rotary_emb.
Unsloth: Compiled function eager_attention_forward.
Unsloth: Compiled function load_balancing_loss_func.
==((====))==  Unsloth 2025.12.7: Fast Gpt_Oss patching. Transformers: 4.56.2.
   \\   /|    NVIDIA GB10. Num GPUs = 1. Max memory: 119.635 GB. Platform: Linux

[huggingface_hub.file_download|INFO]Downloading '.gitattributes' to '/tmp/tmpiq9sx0yq/.cache/huggingface/download/wPaCkH-WbT7GsmxMKKrNZTV4nSM=.a6344aac8c09253b3b630fb776ae94478aa0275b.incomplete'
Download complete. Moving file to /tmp/tmpiq9sx0yq/.gitattributes
[huggingface_hub.file_download|INFO]Download complete. Moving file to /tmp/tmpiq9sx0yq/.gitattributes
[huggingface_hub.file_download|INFO]Downloading 'README.md' to '/tmp/tmpiq9sx0yq/.cache/huggingface/download/Xn7B-BWUGOee2Y6hCZtEhtFu4BE=.e7e1860c72f904dcbe246fc7c83fa8c4e61e014e.incomplete'
Download complete. Moving file to /tmp/tmpiq9sx0yq/README.md
[huggingface_hub.file_download|INFO]Download complete. Moving file to /tmp/tmpiq9sx0yq/README.md
[huggingface_hub.file_download|INFO]Downloading 'config.json' to '/tmp/tmpiq9sx0yq/.cache/huggingface/download/8_PA_wEVGiVa2goH2H4KQOQpvVY=.c0fd5260818f672a9119328c53cc33df84f30698.incomplete'
Download complete. Moving file to /tmp/tmpiq9sx0yq/config.json
[huggingface_hub.file_download|

Unsloth: Offloading embeddings to RAM to save 1.08 GB.
✓ Model loaded successfully!


## Apply LoRA for Efficient RL

Use LoRA to add only 1-5% extra weights, saving 60%+ memory while maintaining accuracy.

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank,
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = lora_rank * 2,  # *2 speeds up training
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

print(f"✓ LoRA applied with rank {lora_rank}")

Unsloth: Making `model.base_model.model.model` require gradients
✓ LoRA applied with rank 16


## Setup Chat Template and Tokenizer

In [4]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "chatml",
)

print("✓ Chat template configured")

Unsloth: Will map <|im_end|> to EOS = <|return|>.


✓ Chat template configured


In [5]:
%pip install openai tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 9.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [openai]2m3/4 [openai]
Note: you may need to restart the kernel to use updated packages.


## Load Tau-Bench Components

Import necessary components from tau-bench for task loading and tool execution.

In [6]:
import json
import logging
import random
from pathlib import Path
from typing import Dict, List, Any, Optional, Tuple
from dataclasses import dataclass, field
from tqdm.auto import tqdm

# Import tau-bench components
from task_tester import Task, ToolCall, ToolExecutor
from data_reader import TauBenchDataReader

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("✓ Tau-bench components loaded")

[matplotlib.font_manager|INFO]generated new fontManager


✓ Tau-bench components loaded


## Define RL Task Format

Convert tau-bench tasks to RL-compatible format with prompts.

In [7]:
@dataclass
class RLTask:
    """RL-compatible task representation"""
    task_id: str
    prompt: str
    expected_actions: List[ToolCall]
    gold_actions_str: str
    metadata: Dict[str, Any] = field(default_factory=dict)
    
    @classmethod
    def from_tau_bench_task(cls, task: Task, system_prompt: str, available_tools: List[Dict]) -> 'RLTask':
        """Convert tau-bench task to RL task format
        
        Creates prompts in plain format (not chat format) for raw text generation.
        The tokenizer's chat template will be applied during tokenization if needed.
        """
        full_prompt = f"{system_prompt}\n\nQuery: {task.query}"
        
        gold_actions_str = json.dumps([
            {"name": action.name, "arguments": action.arguments}
            for action in task.expected_actions
        ])
        
        return cls(
            task_id=task.task_id,
            prompt=full_prompt,
            expected_actions=task.expected_actions,
            gold_actions_str=gold_actions_str,
            metadata=task.metadata
        )

print("✓ RLTask class defined")


✓ RLTask class defined


## Initialize Tool Executor

Set up the tool executor for real tool execution during training.

In [8]:
# Initialize tool executor
envs_path = "envs/retail"
tool_executor = ToolExecutor(envs_path)

available_tools = tool_executor.get_available_tools()
# Build dynamic tools section from available tool schemas
def _format_tool_signature(tool: dict) -> str:
    fn = tool.get('function', {}) or {}
    name = fn.get('name', '')
    params = (fn.get('parameters', {}) or {}).get('properties', {})
    required = list((fn.get('parameters', {}) or {}).get('required', []))
    # Required args first, then optional args
    arg_list = required + [k for k in params.keys() if k not in required]
    return f"- {name}(" + ", ".join(arg_list) + ")"

TOOLS_SECTION = "\n".join([_format_tool_signature(t) for t in available_tools])
print(f"  Available tools: {[t.get('function', {}).get('name') for t in available_tools[:5]]}...")

[task_tester|INFO]Initialized real tool executor with 16 tools


  Available tools: ['calculate', 'cancel_pending_order', 'exchange_delivered_order_items', 'find_user_id_by_email', 'find_user_id_by_name_zip']...


## Define System Prompt

Create the system prompt that guides the model's behavior (following task_tester.py style).

In [9]:
SYSTEM_PROMPT = f"""You are a customer service API. Output ONLY valid JSON.

CRITICAL RULES:
1. Output format: {{"tool_calls": [{{"name": "tool_name", "arguments": {{...}}}}...]}}
2. NEVER output plain text, explanations, or reasoning
3. Use only provided tools and actual data from customer request
4. Use only identifiers present in the request or retrieved via tools
5. Never invent user IDs, order IDs, product IDs, or emails
6. Use tool names and arguments exactly as defined

Available tools:
{TOOLS_SECTION}

FORMAT EXAMPLE (single tool call):
{{"tool_calls": [{{"name": "tool_name", "arguments": {{"arg1": "value1", "arg2": "value2"}}}}]}}

FORMAT EXAMPLE (multiple tool calls):
{{"tool_calls": [{{"name": "tool_1", "arguments": {{"arg": "val"}}}}, {{"name": "tool_2", "arguments": {{"arg": "val"}}}}]}}

ABSOLUTELY NO OTHER OUTPUT ALLOWED.""".strip()

print("✓ System prompt defined (includes available tools)")


✓ System prompt defined (includes available tools)


## Load Tau-Bench Dataset

Load tasks from dataset.jsonl and convert to RL format.

## Quick Health Check

Validate dataset and components before training.

In [10]:
def load_tau_bench_tasks(dataset_path: str, system_prompt: str, available_tools: List[Dict]) -> List[RLTask]:
    """Load and convert tau-bench tasks to RL format"""
    tasks = []
    
    with open(dataset_path, 'r', encoding='utf-8') as f:
        for line_num, line in enumerate(f, 1):
            try:
                data = json.loads(line.strip())
                task = Task.from_dict(data, task_id=f"task_{line_num}")
                rl_task = RLTask.from_tau_bench_task(task, system_prompt, available_tools)
                tasks.append(rl_task)
            except Exception as e:
                logger.warning(f"Failed to load task on line {line_num}: {e}")
                continue
    
    return tasks

# Load tasks
dataset_path = "1k.jsonl"
rl_tasks = load_tau_bench_tasks(dataset_path, SYSTEM_PROMPT, available_tools)

print(f"✓ Loaded {len(rl_tasks)} RL tasks from {dataset_path}")

# CRITICAL VALIDATION: Check if tasks have expected actions
num_no_expected = sum(1 for t in rl_tasks if not t.expected_actions)
if num_no_expected > 0:
    print(f"⚠️ WARNING: {num_no_expected}/{len(rl_tasks)} tasks have NO expected actions!")
    print(f"   This will cause 0 rewards during training. Check dataset schema.")
    if num_no_expected > len(rl_tasks) * 0.5:
        raise ValueError(f"❌ More than 50% of tasks ({num_no_expected}/{len(rl_tasks)}) have no expected actions! Training will fail. Check dataset format.")

print(f"\n✓ Validation passed: {len(rl_tasks) - num_no_expected}/{len(rl_tasks)} tasks have expected actions")
print(f"\nExample task:")
print(f"  ID: {rl_tasks[0].task_id}")
print(f"  Prompt (first 200 chars): {rl_tasks[0].prompt[:200]}...")
print(f"  Expected actions: {len(rl_tasks[0].expected_actions)}")

✓ Loaded 2300 RL tasks from 1k.jsonl

✓ Validation passed: 2300/2300 tasks have expected actions

Example task:
  ID: task_1
  Prompt (first 200 chars): You are a customer service API. Output ONLY valid JSON.

CRITICAL RULES:
1. Output format: {"tool_calls": [{"name": "tool_name", "arguments": {...}}...]}
2. NEVER output plain text, explanations, or r...
  Expected actions: 3


## Define Reward Function

Create a reward function that evaluates model responses based on:
- **Action Correctness**: +1.0 per correct action, -0.5 per incorrect
- **Execution Success**: +0.5 for successful tool execution
- **Ordering**: +0.5 for correct action sequence

In [11]:
def mask_for_log(obj):
    """Simple masking helper for logging to avoid leaking emails and long strings."""
    def _mask_val(v):
        try:
            if isinstance(v, str):
                if '@' in v:
                    parts = v.split('@', 1)
                    local = parts[0]
                    domain = parts[1]
                    if len(local) > 2:
                        return local[0] + '***' + local[-1] + '@' + domain
                    return '***@' + domain
                if len(v) > 120:
                    return v[:100] + '...'
                return v
            if isinstance(v, dict):
                return {k: _mask_val(val) for k, val in v.items()}
            if isinstance(v, list):
                return [_mask_val(x) for x in v]
            if isinstance(v, set):
                return [_mask_val(x) for x in sorted(v)]
            return v
        except Exception:
            return '<masked>'
    return _mask_val(obj)

In [12]:
def _get_required_args(tool_name: str) -> list:
    """Get required arguments for a tool."""
    required_args_map = {
        'return_delivered_order_items': ['order_id', 'item_ids', 'payment_method_id'],
        'exchange_delivered_order_items': ['order_id', 'item_ids', 'new_item_ids', 'payment_method_id'],
        'cancel_pending_order': ['order_id', 'reason'],
        'find_user_id_by_email': ['email'],
        'find_user_id_by_name_zip': ['first_name', 'last_name', 'zip'],
        'get_order_details': ['order_id'],
        'get_user_details': ['user_id'],
        'get_product_details': ['product_id'],
        'modify_pending_order_address': ['order_id', 'address1', 'city', 'state', 'zip'],  # address2, country optional
        'modify_pending_order_items': ['order_id', 'item_ids', 'new_item_ids', 'payment_method_id'],
        'modify_pending_order_payment': ['order_id', 'payment_method_id'],
        'modify_user_address': ['user_id', 'address1', 'city', 'state', 'zip'],  # address2, country optional
        'transfer_to_human_agents': ['reason'],
    }
    return required_args_map.get(tool_name, [])

In [13]:
class TauBenchRewardFunction:
    """
    Tool-level cumulative reward for true GRPO (not just terminal).
    
    Reward shaping:
    - Per-action credit: +1.0 per correct tool name
    - Per-action args: +0.5 per correct argument set
    - Ordering bonus: +0.5 if full sequence correct
    - Penalties: -0.3 per wrong action, -0.4 per missing required arg
    """
    
    def __init__(self, tool_executor: ToolExecutor):
        self.tool_executor = tool_executor
        self.parse_failure_count = 0
        self.last_parse_failures = 0
    

    def calculate_reward(self, response: str, task: RLTask) -> Tuple[float, Dict[str, Any]]:
        """Calculate tool-level cumulative reward (TRUE GRPO)."""
        
        self.task_count = getattr(self, 'task_count', 0) + 1
        
        if not task.expected_actions:
            return 0.0, {'error': 'no_expected_actions'}
        
        try:
            # Parse response
            parsed_actions = self._parse_response_to_actions(response)
            parsed_actions = self._sanitize_parsed_actions(parsed_actions)
            
            # Tool-level cumulative reward (not just terminal)
            tool_rewards = []
            completeness_penalty = 0.0
            missing_args_details = []
            
            for action in parsed_actions:
                required_args = _get_required_args(action.name)
                missing = [arg for arg in required_args if arg not in action.arguments or not action.arguments[arg]]
                
                if missing:
                    penalty = len(missing) * -0.4
                    completeness_penalty += penalty
                    missing_args_details.append({
                        'tool': action.name,
                        'missing': missing,
                        'penalty': penalty
                    })
            
            # FIX #2: Remove auto-filling from reward function
            # Only normalize args, DO NOT fill missing args
            # This forces the model to learn to generate complete args
            if parsed_actions and self.tool_executor:
                try:
                    for act in parsed_actions:
                        args = act.arguments if isinstance(act.arguments, dict) else {}
                        try:
                            from utils.arg_normalizer import normalize_action_arguments
                            args = normalize_action_arguments(act.name, args)
                            act.arguments = args
                        except:
                            pass
                except Exception as e:
                    logger.debug(f"Normalization error: {e}")
            
            # Calculate per-action metrics
            metrics = self._calculate_improved_action_metrics(parsed_actions, task.expected_actions)
            
            # Tool-level cumulative rewards (NOT just terminal)
            reward = 0.0
            reward += metrics['correct_action_names'] * 1.0  # per correct tool
            reward += metrics['correct_arguments'] * 0.5     # per correct args
            reward += 0.5 if metrics['correct_ordering'] else 0.0  # ordering bonus
            reward += metrics['incorrect_actions'] * -0.3
            reward += completeness_penalty
            
            # Clip to prevent explosion (NO normalize to [0,1] - let GRPOTrainer do advantage computation)
            reward = max(min(reward, 10.0), -10.0)
            
            # Track stats
            if not hasattr(self, 'total_rewards'):
                self.total_rewards = []
                self.zero_reward_count = 0
            self.total_rewards.append(reward)
            if reward <= 0.0:
                self.zero_reward_count += 1
            
            # Metrics
            metrics.update({
                'reward': reward,
                'completeness_penalty': completeness_penalty,
                'missing_args_details': missing_args_details,
                'parse_failures_total': self.parse_failure_count,
                'task_count': self.task_count
            })
            
            # Log first few
            if self.task_count <= 5:
                logger.info(f"Task {task.task_id} - Reward: {reward:.3f}")
                logger.info(f"  Parsed: {[a.name for a in parsed_actions]}")
                logger.info(f"  Expected: {[a.name for a in task.expected_actions]}")
                logger.info(f"  Metrics: correct_names={metrics['correct_action_names']}/{metrics['total_expected']}, "
                        f"correct_args={metrics['correct_arguments']}, completeness_penalty={completeness_penalty:.2f}")
            
            return reward, metrics
            
        except Exception as e:
            logger.error(f"Reward error: {e}", exc_info=True)
            self.parse_failure_count += 1
            return -0.5, {'error': str(e)}

    
    def _calculate_improved_action_metrics(self, parsed: List[ToolCall], expected: List[ToolCall]) -> Dict[str, Any]:
        """Calculate improved action matching metrics with flexible matching."""
        
        def normalize_name(name: str) -> str:
            return str(name).lower().strip().replace('-', '_').replace(' ', '_')
        
        parsed_names = [normalize_name(a.name) for a in parsed]
        expected_names = [normalize_name(a.name) for a in expected]
        
        correct_action_names = 0
        correct_arguments = 0
        matched_expected_indices = set()
        matched_parsed_indices = set()
        
        # Match actions by name
        for i, exp_action in enumerate(expected):
            exp_name = normalize_name(exp_action.name)
            
            for j, parsed_action in enumerate(parsed):
                if j in matched_parsed_indices:
                    continue
                
                if exp_name == normalize_name(parsed_action.name):
                    correct_action_names += 1
                    matched_expected_indices.add(i)
                    matched_parsed_indices.add(j)
                    
                    # Check arguments with fuzzy matching
                    if self._arguments_match_fuzzy(parsed_action.arguments, exp_action.arguments, threshold=0.6):
                        correct_arguments += 1
                    break
        
        incorrect_actions = len(parsed) - len(matched_parsed_indices)
        correct_ordering = (len(parsed) == len(expected) and 
                          correct_action_names == len(expected) and 
                          parsed_names == expected_names)
        
        precision = correct_action_names / len(parsed) if parsed else 0.0
        recall = correct_action_names / len(expected) if expected else 0.0
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0
        
        return {
            'correct_action_names': correct_action_names,
            'correct_arguments': correct_arguments,
            'incorrect_actions': incorrect_actions,
            'correct_ordering': correct_ordering,
            'total_expected': len(expected),
            'total_parsed': len(parsed),
            'precision': precision,
            'recall': recall,
            'f1': f1
        }
    
    def _arguments_match_fuzzy(self, parsed_args: Dict[str, Any], expected_args: Dict[str, Any], threshold: float = 0.6) -> bool:
        """Check if arguments match with fuzzy comparison."""
        if not expected_args:
            return True
        if not parsed_args:
            return False
        
        matches = 0
        for key, expected_val in expected_args.items():
            parsed_val = None
            
            # Try exact key
            if key in parsed_args:
                parsed_val = parsed_args[key]
            # Try variations
            else:
                for pk in parsed_args.keys():
                    if pk.replace('_', '').lower() == key.replace('_', '').lower():
                        parsed_val = parsed_args[pk]
                        break
            
            if parsed_val is not None and self._values_match_fuzzy(parsed_val, expected_val):
                matches += 1
        
        return (matches / len(expected_args)) >= threshold
    
    def _values_match_fuzzy(self, val1: Any, val2: Any) -> bool:
        """Check if values match with normalization."""
        if val1 is None or val2 is None:
            return val1 == val2
        
        str1, str2 = str(val1).strip(), str(val2).strip()
        
        # Case insensitive
        if str1.lower() == str2.lower():
            return True
        
        # Order IDs
        if any(c in str1.lower() + str2.lower() for c in ['#', 'w']) or str1.isdigit() or str2.isdigit():
            norm1 = self._normalize_order_id_fuzzy(str1)
            norm2 = self._normalize_order_id_fuzzy(str2)
            if norm1 == norm2:
                return True
        
        # Emails
        if '@' in str1 or '@' in str2:
            return str1.lower() == str2.lower()
        
        # Numeric
        try:
            return abs(float(str1.replace(',', '')) - float(str2.replace(',', ''))) < 0.01
        except:
            pass
        
        return False
    
    def _normalize_order_id_fuzzy(self, order_id: str) -> str:
        """Normalize order ID."""
        import re
        match = re.search(r'(\d+)', str(order_id))
        return f"#W{match.group(1)}" if match else str(order_id).strip().upper()
    
    def get_statistics(self) -> Dict[str, Any]:
        """Get training statistics."""
        if not hasattr(self, 'total_rewards'):
            self.total_rewards = []
            self.zero_reward_count = 0
        
        avg_reward = sum(self.total_rewards) / len(self.total_rewards) if self.total_rewards else 0.0
        task_count = getattr(self, 'task_count', 0)
        
        return {
            'task_count': task_count,
            'average_reward': avg_reward,
            'zero_reward_count': self.zero_reward_count,
            'zero_reward_rate': self.zero_reward_count / task_count if task_count > 0 else 0,
            'parse_failures': self.parse_failure_count,
            'parse_failure_rate': self.parse_failure_count / task_count if task_count > 0 else 0
        }

    def _extract_customer_info_from_prompt(self, prompt: str) -> Dict[str, Any]:
        """Extract customer information from task prompt text."""
        info = {}
        
        if not prompt:
            return info
        
        # Import normalizer
        try:
            from task_generator import _normalize_order_id_global
        except ImportError:
            def _normalize_order_id_global(oid):
                oid = str(oid).upper()
                if not oid.startswith('#'):
                    oid = '#' + oid
                if not oid.startswith('#W'):
                    oid = '#W' + oid.replace('#', '')
                return oid
        import re
        # Extract email
        email_match = re.search(r'[\w\.-]+@[\w\.-]+', prompt)
        if email_match:
            info['email'] = email_match.group(0)
        
        # Extract order ID
        order_match = re.search(r'#?W\d{4,}', prompt, re.I)
        if order_match:
            info['order_id'] = _normalize_order_id_global(order_match.group(0))
        
        # Extract zip code
        zip_match = re.search(r'\b(\d{5})\b', prompt)
        if zip_match:
            info['zip'] = zip_match.group(1)
        
        # Extract name (First Last pattern)
        name_match = re.search(r'\b([A-Z][a-z]+)\s+([A-Z][a-z]+)\b', prompt)
        if name_match:
            info['first_name'] = name_match.group(1)
            info['last_name'] = name_match.group(2)
        
        return info


    def _tool_expects_arg(self, tool_name: str, arg_name: str) -> bool:
        """Check if a tool's signature expects a given argument."""
        try:
            if not self.tool_executor or not hasattr(self.tool_executor, '_tools'):
                return True
            
            tool_data = self.tool_executor._tools.get(tool_name)
            if not tool_data:
                return True
            
            tool_cls = tool_data.get('class')
            if not tool_cls:
                return True
            
            import inspect
            sig = inspect.signature(tool_cls.invoke)
            params = list(sig.parameters.keys())
            
            # Remove 'self' and 'data' if present
            params = [p for p in params if p not in ('self', 'data')]
            
            return arg_name in params
        except Exception:
            return True

    def _parse_response_to_actions(self, response: str) -> List[ToolCall]:
        """Parse model response to extract tool calls."""
        import re, ast

        actions = []
        self.last_parse_failures = 0

        try:
            # Find balanced JSON-like substrings
            def _find_json_substrings(text: str):
                subs = []
                stack = []
                start = None
                pairs = {'{': '}', '[': ']'}
                for i, ch in enumerate(text):
                    if ch in pairs:
                        if start is None:
                            start = i
                        stack.append(ch)
                    elif ch in pairs.values() and stack:
                        open_ch = stack[-1]
                        if pairs.get(open_ch) == ch:
                            stack.pop()
                            if not stack and start is not None:
                                subs.append(text[start:i+1])
                                start = None
                return subs

            json_candidates = _find_json_substrings(response)

            parsed_any = False
            for json_str in json_candidates:
                try:
                    data = json.loads(json_str)

                except json.JSONDecodeError:
                    try:
                        data = ast.literal_eval(json_str)
                        logger.debug("Parsed JSON-like candidate with ast.literal_eval")
                    except Exception:
                        logger.debug("Invalid JSON candidate skipped")
                        continue

                # Normalize container formats
                if isinstance(data, dict):
                    if 'tool_calls' in data:
                        tool_calls = data['tool_calls']
                    elif 'actions' in data:
                        tool_calls = data['actions']
                    elif 'response' in data and isinstance(data['response'], dict) and 'tool_calls' in data['response']:
                        tool_calls = data['response']['tool_calls']
                    else:
                        tool_calls = [data]
                elif isinstance(data, list):
                    tool_calls = data
                else:
                    tool_calls = [data]

                for call_data in tool_calls:
                    if not isinstance(call_data, dict):
                        continue

                    name = (
                        call_data.get('name') or
                        (call_data.get('function') and call_data['function'].get('name')) or
                        call_data.get('tool') or
                        call_data.get('tool_name') or
                        ''
                    )

                    if 'arguments' in call_data and isinstance(call_data['arguments'], dict):
                        arguments = call_data['arguments']
                    elif 'args' in call_data and isinstance(call_data['args'], dict):
                        arguments = call_data['args']
                    else:
                        non_arg_keys = {'name', 'function', 'tool', 'tool_name', 'result', 'success', 'execution_time'}
                        potential_args = {k: v for k, v in call_data.items() if k not in non_arg_keys}
                        if potential_args:
                            arguments = potential_args
                            logger.debug(f"Treated top-level dict as arguments for call: keys={list(arguments.keys())}")
                        else:
                            arguments = {}

                    if not name and isinstance(arguments, dict) and arguments:
                        inferred = None
                        try:
                            inferred = self._infer_tool_from_argument_keys(set(arguments.keys()))
                        except Exception:
                            inferred = None

                        if inferred:
                            name = inferred
                            logger.info(f"Inferred tool name '{name}' from argument keys: {list(arguments.keys())}")
                        else:
                            logger.warning(f"Skipping malformed tool call (missing name)")
                            continue

                    if not name:
                        logger.warning(f"Skipping malformed tool call (missing name)")
                        continue

                    actions.append(ToolCall(name=name, arguments=arguments))

                if actions:
                    parsed_any = True
                    break

            if not parsed_any:
                if json_candidates:
                    self.last_parse_failures += 1
                    self.parse_failure_count += 1
                    logger.debug(f"JSON candidates present but none parsed successfully")
                else:
                    logger.debug("No JSON-like substring found in response")

            # Fallback: textual pattern parsing
            if not actions:
                pattern = r"(\w+)\((.*?)\)"
                matches = re.findall(pattern, response, re.DOTALL)

                for tool_name, args_str in matches:
                    try:
                        args = {}
                        args_str = args_str.strip()
                        if args_str:
                            parts = re.split(r',\s*(?![^()]*\))', args_str)
                            for part in parts:
                                if '=' in part:
                                    k, v = part.split('=', 1)
                                    k = k.strip()
                                    v = v.strip()
                                    if (v.startswith('"') and v.endswith('"')) or (v.startswith("'") and v.endswith("'")):
                                        v = v[1:-1]
                                    args[k] = v
                        actions.append(ToolCall(name=tool_name, arguments=args))
                    except Exception:
                        continue

        except Exception as e:
            logger.warning(f"Unexpected error parsing response: {e}")
            self.last_parse_failures += 1
            self.parse_failure_count += 1

        return actions
    

    def _extract_order_from_text(self, text: str) -> Optional[str]:
        """Extract order ID or zip code from text."""
        import re
        
        if not isinstance(text, str):
            return None
        
        # Try to extract order ID first
        m = re.search(r"#W\d{4,}", text)
        if m:
            return m.group(0)
        
        # Fallback to zip code
        z = re.search(r"(\d{5})", text)
        return z.group(1) if z else None


    def _infer_tool_from_argument_keys(self, arg_keys: set) -> Optional[str]:
        """Infer the best-matching tool name from argument keys."""
        try:
            if not self.tool_executor or not hasattr(self.tool_executor, '_tools'):
                return None

            best_name = None
            best_overlap = 0
            for tool_name, tool_data in self.tool_executor._tools.items():
                try:
                    tool_cls = tool_data.get('class')
                    import inspect
                    sig = inspect.signature(tool_cls.invoke)
                    params = list(sig.parameters.keys())
                    if params and params[0] == 'self':
                        params = params[1:]
                    if params and params[0] == 'data':
                        params = params[1:]
                    expected = set(params)
                    overlap = len(arg_keys & expected)
                    if overlap > best_overlap:
                        best_overlap = overlap
                        best_name = tool_name
                except Exception:
                    continue

            if best_overlap >= 2:
                return best_name
            return None
        except Exception:
            return None

    def _sanitize_parsed_actions(self, actions: List[ToolCall]) -> List[ToolCall]:
        """Sanitize parsed actions."""
        def _sanitize_value(v):
            if isinstance(v, set):
                return [_sanitize_value(x) for x in v]
            if isinstance(v, tuple):
                return [_sanitize_value(x) for x in v]
            if isinstance(v, list):
                return [_sanitize_value(x) for x in v]
            if isinstance(v, dict):
                return {str(k): _sanitize_value(val) for k, val in v.items()}
            return v

        for act in actions:
            if not isinstance(act.arguments, dict):
                try:
                    act.arguments = dict(act.arguments)
                except Exception:
                    act.arguments = {}
            # Normalize singular keys
            if 'item_id' in act.arguments and 'item_ids' not in act.arguments:
                act.arguments['item_ids'] = _sanitize_value(act.arguments.pop('item_id'))
                if not isinstance(act.arguments['item_ids'], list):
                    act.arguments['item_ids'] = [act.arguments['item_ids']]
            # Handle name -> first/last
            if act.name == 'find_user_id_by_name_zip' and 'name' in act.arguments:
                full = act.arguments.get('name')
                if isinstance(full, str):
                    parts = full.strip().split()
                    if len(parts) >= 2:
                        act.arguments.setdefault('first_name', parts[0])
                        act.arguments.setdefault('last_name', parts[-1])
            # sanitize all values
            for k, v in list(act.arguments.items()):
                act.arguments[k] = _sanitize_value(v)

            # Apply normalizer
            from utils.arg_normalizer import normalize_action_arguments
            act.arguments = normalize_action_arguments(act.name, act.arguments)

        return actions

print("✓ TauBenchRewardFunction with tool-level cumulative rewards")

✓ TauBenchRewardFunction with tool-level cumulative rewards


In [14]:
# Filter/fix tasks to avoid missing required arguments in gold actions
from typing import Tuple, List, Any
import re

def _validate_address_fields(arguments: dict) -> bool:
	"""Validate that address fields are properly filled and not mixed format."""
	address_fields = ['address1', 'city', 'state', 'zip']

	# Check if this action involves address modification
	has_address = any(field in arguments for field in address_fields)
	if not has_address:
		return True

	# If address1 exists, check for mixed format (contains city/zip in same field)
	address1 = arguments.get('address1', '')
	if address1:
		# Check for patterns like "City, Zip XXXXX" or "City, State ZIP" in address1
		mixed_format_patterns = [
			r',\s*Zip\s+\d{5}',  # ", Zip 19117"
			r',\s*[A-Z]{2}\s+\d{5}',  # ", PA 19117"
			r'\d{5}\s*$',  # ends with zip code
		]
		for pattern in mixed_format_patterns:
			if re.search(pattern, address1, re.IGNORECASE):
				# Mixed format detected, check if other fields are empty
				if not arguments.get('city') or not arguments.get('state') or not arguments.get('zip'):
					return False

	# Validate all address fields are non-empty if address modification is required
	for field in address_fields:
		if field in arguments:
			value = arguments[field]
			if not value or (isinstance(value, str) and value.strip() == ''):
				return False

	# Validate zip code format (5 digits)
	if 'zip' in arguments:
		zip_code = str(arguments['zip']).strip()
		if not re.match(r'^\d{5}$', zip_code):
			return False

	# Validate state format (2 uppercase letters)
	if 'state' in arguments:
		state = str(arguments['state']).strip()
		if not re.match(r'^[A-Z]{2}$', state):
			return False

	return True


def _extract_product_ids_from_query(query: str) -> set:
	"""Extract all product IDs mentioned in the query."""
	# Match patterns like: (1075968781), ID: 1075968781, #1075968781, etc.
	patterns = [
		r'\((\d{10})\)',  # (1075968781)
		r'ID[:\s]+(\d{10})',  # ID: 1075968781
		r'#(\d{10})',  # #1075968781
		r'\b(\d{10})\b',  # standalone 10-digit number
	]

	product_ids = set()
	for pattern in patterns:
		matches = re.findall(pattern, query, re.IGNORECASE)
		product_ids.update(matches)

	return product_ids


def _validate_item_ids_match_query(action_name: str, arguments: dict, query: str) -> bool:
	"""Validate that item_ids in action match product IDs mentioned in query."""
	# Only validate for actions that involve item_ids
	item_actions = [
		'return_delivered_order_items',
		'exchange_delivered_order_items',
		'modify_pending_order_items'
	]

	if action_name not in item_actions:
		return True

	# Extract product IDs from query
	query_product_ids = _extract_product_ids_from_query(query)
	if not query_product_ids:
		# If no product IDs in query, we can't validate - be lenient
		return True

	# Get item_ids from action arguments
	item_ids = arguments.get('item_ids', [])
	if not isinstance(item_ids, list):
		item_ids = [item_ids] if item_ids else []

	# Convert to strings for comparison
	item_ids_str = set(str(item_id) for item_id in item_ids if item_id)

	# Check if any item_id matches any product_id from query
	if item_ids_str and not item_ids_str.intersection(query_product_ids):
		return False

	return True


def _validate_email_in_query(action_name: str, arguments: dict, query: str) -> bool:
	"""Validate that email exists in query when action requires it."""
	# Only validate for actions that require email
	if action_name != 'find_user_id_by_email':
		return True

	# Check if email is required
	if 'email' not in arguments:
		return False

	# Check if query contains any email address
	email_pattern = r'[\w\.-]+@[\w\.-]+'
	email_in_query = re.search(email_pattern, query)

	if not email_in_query:
		# Email required but not found in query
		return False

	return True


def _validate_field_format(field_name: str, field_value: Any) -> bool:
	"""Validate specific field formats."""
	if field_value is None:
		return False

	# Convert to string for validation
	value_str = str(field_value).strip()

	if not value_str or value_str == '':
		return False

	# Specific format validations
	if field_name == 'email':
		return bool(re.match(r'^[\w\.-]+@[\w\.-]+\.\w+$', value_str))

	elif field_name == 'zip':
		return bool(re.match(r'^\d{5}$', value_str))

	elif field_name == 'state':
		return bool(re.match(r'^[A-Z]{2}$', value_str))

	elif field_name == 'order_id':
		# Order ID should start with # or W and contain digits
		return bool(re.match(r'^#?W\d+$', value_str, re.IGNORECASE))

	elif field_name in ['user_id', 'product_id']:
		# Should be numeric
		return value_str.isdigit()

	elif field_name in ['first_name', 'last_name', 'city']:
		# Should contain letters and be reasonable length
		return len(value_str) >= 2 and bool(re.search(r'[A-Za-z]', value_str))

	elif field_name == 'reason':
		# Should be non-empty string with reasonable length
		return len(value_str) >= 3

	elif field_name in ['item_ids', 'new_item_ids']:
		# Should be a list
		if not isinstance(field_value, list):
			return False
		# Each item should be valid (non-empty string)
		return all(str(item).strip() for item in field_value)

	# Default: non-empty
	return True


def filter_and_fix_tasks(tasks: List[RLTask], tool_executor: ToolExecutor) -> Tuple[List[RLTask], dict]:
	"""Filter tasks and fix missing required args (uses local fixer, not global reward_function)."""
	# Create local fixer for extracting customer info - doesn't depend on reward_function
	class LocalTaskFixer:
		def _extract_customer_info_from_prompt(self, prompt: str) -> dict:
			"""Extract known customer info patterns from prompt text."""
			info = {}

			# Email
			email_match = re.search(r'([\w\.-]+@[\w\.-]+\.\w+)', prompt)
			if email_match:
				info['email'] = email_match.group(1)

			# Names
			name_patterns = [
				r'(?:name|person|customer|user)\s+(?:is|of|for)\s+([A-Z][a-z]+)\s+([A-Z][a-z]+)',
				r'(?:this is|I am|I\'m)\s+([A-Z][a-z]+)\s+([A-Z][a-z]+)',
				r'([A-Z][a-z]+)\s+([A-Z][a-z]+)(?:\s+from|\s+and|\b)',
			]
			for pattern in name_patterns:
				match = re.search(pattern, prompt)
				if match:
					info['first_name'] = match.group(1)
					info['last_name'] = match.group(2)
					break

			# Zip
			zip_match = re.search(r'\b(\d{5})\b', prompt)
			if zip_match:
				info['zip'] = zip_match.group(1)

			# Order ID
			order_match = re.search(r'#?W\d{4,}', prompt, re.I)
			if order_match:
				info['order_id'] = order_match.group(0)

			# User ID
			user_match = re.search(r'user[_\s]?(?:id|ID|#)?[:\s]+(\d+)', prompt, re.I)
			if user_match:
				info['user_id'] = user_match.group(1)

			return info

		def _extract_order_from_text(self, text: str) -> str:
			"""Extract order ID from text."""
			match = re.search(r'#?W\d{4,}', text, re.I)
			return match.group(0) if match else None

	fixer = LocalTaskFixer()
	kept: List[RLTask] = []
	stats = {
		'total': len(tasks),
		'dropped_no_expected': 0,
		'dropped_missing_required': 0,
		'dropped_invalid_address': 0,
		'dropped_item_id_mismatch': 0,
		'dropped_missing_email': 0,
		'dropped_invalid_format': 0,
		'fixed_args': 0,
	}

	for t in tasks:
		if not t.expected_actions:
			stats['dropped_no_expected'] += 1
			continue

		prompt_info = fixer._extract_customer_info_from_prompt(t.prompt)
		expected_any_missing = False
		validation_failed = False
		validation_reason = None
		fixed_count = 0

		# Build name→args map of gold actions and attempt conservative fills
		for a in t.expected_actions:
			reqs = _get_required_args(a.name)
			if not isinstance(a.arguments, dict):
				a.arguments = {}
			# Fill from prompt_info when safe
			for k in reqs:
				if k not in a.arguments or a.arguments.get(k) in (None, ""):
					v = prompt_info.get(k)
					if v is not None:
						a.arguments[k] = v
						fixed_count += 1

			# Heuristics for common cases
			if a.name in ("exchange_delivered_order_items", "return_delivered_order_items"):
				if not a.arguments.get("order_id"):
					# Try to extract from prompt
					oid = fixer._extract_order_from_text(t.prompt)
					if oid:
						a.arguments["order_id"] = oid
						fixed_count += 1
				# Ensure lists for item_ids/new_item_ids
				for list_key in ("item_ids", "new_item_ids"):
					if list_key in reqs and not isinstance(a.arguments.get(list_key), list):
						if a.arguments.get(list_key) is None:
							a.arguments[list_key] = []
							fixed_count += 1
						else:
							a.arguments[list_key] = [a.arguments[list_key]]
							fixed_count += 1

			if a.name == "find_user_id_by_name_zip":
				# If only 'name' present, split into first/last
				full = a.arguments.get('name')
				if isinstance(full, str) and (not a.arguments.get('first_name') or not a.arguments.get('last_name')):
					parts = full.strip().split()
					if len(parts) >= 2:
						a.arguments.setdefault('first_name', parts[0])
						a.arguments.setdefault('last_name', parts[-1])
						fixed_count += 2

			# === NEW VALIDATION CHECKS ===

			# 1. Validate address fields for completeness and format
			if not _validate_address_fields(a.arguments):
				validation_failed = True
				validation_reason = 'invalid_address'
				break

			# 2. Validate item_ids match product IDs in query
			if not _validate_item_ids_match_query(a.name, a.arguments, t.prompt):
				validation_failed = True
				validation_reason = 'item_id_mismatch'
				break

			# 3. Validate email exists in query when required
			if not _validate_email_in_query(a.name, a.arguments, t.prompt):
				validation_failed = True
				validation_reason = 'missing_email'
				break

			# 4. Validate all required field formats
			for req_field in reqs:
				if req_field in a.arguments:
					if not _validate_field_format(req_field, a.arguments[req_field]):
						validation_failed = True
						validation_reason = 'invalid_format'
						break

			if validation_failed:
				break

			# Check for remaining missing required args
			still_missing = [k for k in reqs if not a.arguments.get(k)]
			if still_missing:
				expected_any_missing = True

		# Skip task if validation failed
		if validation_failed:
			if validation_reason == 'invalid_address':
				stats['dropped_invalid_address'] += 1
			elif validation_reason == 'item_id_mismatch':
				stats['dropped_item_id_mismatch'] += 1
			elif validation_reason == 'missing_email':
				stats['dropped_missing_email'] += 1
			elif validation_reason == 'invalid_format':
				stats['dropped_invalid_format'] += 1
			continue

		if expected_any_missing:
			stats['dropped_missing_required'] += 1
			continue

		if fixed_count:
			stats['fixed_args'] += fixed_count
		kept.append(t)

	return kept, stats


# Apply filtering/fixing
rl_tasks, fstats = filter_and_fix_tasks(rl_tasks, tool_executor)
print("\n✅ Task filtering complete (enhanced with comprehensive validation)")
print(f"  Total:                                    {fstats['total']}")
print(f"  Kept:                                     {len(rl_tasks)}")
print(f"  ─────────────────────────────────────────")
print(f"  Dropped (no expected):                    {fstats['dropped_no_expected']}")
print(f"  Dropped (missing required after fix):     {fstats['dropped_missing_required']}")
print(f"  Dropped (invalid address format):         {fstats['dropped_invalid_address']}")
print(f"  Dropped (item_id mismatch):               {fstats['dropped_item_id_mismatch']}")
print(f"  Dropped (email not in query):             {fstats['dropped_missing_email']}")
print(f"  Dropped (invalid field format):           {fstats['dropped_invalid_format']}")
print(f"  ─────────────────────────────────────────")
print(f"  Fixed arguments:                          {fstats['fixed_args']}")
print(f"  Keep rate:                                {len(rl_tasks)/fstats['total']*100:.1f}%")



✅ Task filtering complete (enhanced with comprehensive validation)
  Total:                                    2300
  Kept:                                     1323
  ─────────────────────────────────────────
  Dropped (no expected):                    0
  Dropped (missing required after fix):     517
  Dropped (invalid address format):         76
  Dropped (item_id mismatch):               242
  Dropped (email not in query):             0
  Dropped (invalid field format):           142
  ─────────────────────────────────────────
  Fixed arguments:                          0
  Keep rate:                                57.5%


In [15]:
from task_generator import _normalize_order_id_global
import re
def extract_actions_from_reasoning(response: str, task) -> str:
    """Extract tool calls from reasoning text like 'We need to authenticate with find_user_id_by_email'.
    
    This converter handles cases where the model outputs reasoning instead of JSON.
    Pattern examples:
    - "We need to authenticate: find_user_id_by_email with john@example.com"
    - "Cancel order. Use cancel_pending_order with order_id #W123"
    - "Find user by name Chen Moore and zip 91087"
    - "We can use find_user_id_by_name_zip with first_name Yusuf, last_name Khan, zip?"
    """    
    tool_calls = []
    
    # Pattern 1: "find_user_id_by_email with john@example.com" or similar
    email_match = re.search(r'find_user_id_by_email.*?(\S+@\S+)', response)
    if email_match:
        email = email_match.group(1)
        tool_calls.append({
            'name': 'find_user_id_by_email',
            'arguments': {'email': email}
        })
    
    # Pattern 2: "find_user_id_by_name_zip" - IMPROVED to handle partial data
    if 'find_user_id_by_name_zip' in response or 'name' in response.lower():
        # Extract name with quoted patterns: first_name "Yusuf", last_name "Khan"
        name_match = re.search(r'first_name\s+["\']?([A-Z][a-z]+)["\']?\s*,?\s*last_name\s+["\']?([A-Z][a-z]+)["\']?|([A-Z][a-z]+)\s+([A-Z][a-z]+)', response)
        # Extract zip
        zip_match = re.search(r'\b(\d{5})\b', response)
        
        # Also check in task context if zip not found in response
        if not zip_match and task:
            prompt_text = getattr(task, 'prompt', '') or getattr(task, 'query', '')
            if prompt_text:
                zip_match = re.search(r'\b(\d{5})\b', prompt_text)
        
        if name_match:
            # Extract names from regex groups (handles multiple patterns)
            first = name_match.group(1) if name_match.group(1) else (name_match.group(3) if name_match.group(3) else None)
            last = name_match.group(2) if name_match.group(2) else (name_match.group(4) if name_match.group(4) else None)
            
            if first and last:
                args = {
                    'first_name': first,
                    'last_name': last
                }
                if zip_match:
                    args['zip'] = zip_match.group(1)
                
                # Only add if we have at least first_name and last_name
                if 'find_user_id_by_name_zip' not in [tc['name'] for tc in tool_calls]:
                    tool_calls.append({
                        'name': 'find_user_id_by_name_zip',
                        'arguments': args
                    })
    
    # Pattern 3: "cancel_pending_order" or "cancel order" + order ID
    if 'cancel' in response.lower():
        order_match = re.search(r'#?W\d{4,}', response, re.I)
        if order_match:
            oid = _normalize_order_id_global(order_match.group(0))
            if 'cancel_pending_order' not in [tc['name'] for tc in tool_calls]:
                tool_calls.append({
                    'name': 'cancel_pending_order',
                    'arguments': {'order_id': oid}
                })
    
    # Pattern 4: "get_order_details" or "order details"
    if 'get_order_details' in response or ('order' in response.lower() and ('detail' in response.lower() or 'info' in response.lower())):
        order_match = re.search(r'#?W\d{4,}', response, re.I)
        if order_match:
            oid = _normalize_order_id_global(order_match.group(0))
            if 'get_order_details' not in [tc['name'] for tc in tool_calls]:
                tool_calls.insert(max(0, 1), {  # Insert after auth if possible
                    'name': 'get_order_details',
                    'arguments': {'order_id': oid}
                })
    
    # Pattern 5: return/exchange items
    if 'return' in response.lower() or 'exchange' in response.lower():
        order_match = re.search(r'#?W\d{4,}', response, re.I)
        if order_match:
            oid = _normalize_order_id_global(order_match.group(0))
            action_name = 'exchange_delivered_order_items' if 'exchange' in response.lower() else 'return_delivered_order_items'
            if action_name not in [tc['name'] for tc in tool_calls]:
                tool_calls.append({
                    'name': action_name,
                    'arguments': {'order_id': oid, 'item_ids': []}
                })
    
    # Pattern 6: transfer to human agents - ONLY if no action items found
    if 'transfer' in response.lower() or 'human' in response.lower() or 'escalate' in response.lower():
        # Only add if we haven't found better alternatives (no actual action items)
        if not any(tc['name'].startswith(('cancel', 'return', 'exchange', 'modify')) for tc in tool_calls):
            if 'transfer_to_human_agents' not in [tc['name'] for tc in tool_calls]:
                reason = 'Unable to process request'
                # Try to extract why
                if 'missing' in response.lower():
                    reason = 'Missing required information'
                elif "don't" in response.lower() or 'cannot' in response.lower():
                    reason = 'Cannot proceed without additional information'
                tool_calls.append({
                    'name': 'transfer_to_human_agents',
                    'arguments': {'reason': reason}
                })
    
    if tool_calls:
        logger.debug(f"Extracted {len(tool_calls)} actions from reasoning text")
        return json.dumps({'tool_calls': tool_calls})
    
    return None


In [16]:
import json, re, ast

def normalize_response_to_json(response: str, task) -> str:
    """Normalize response to JSON format with tool_calls.
    
    Multi-stage fallback pipeline:
    1. Try to parse as valid JSON with 'tool_calls' key
    2. Try ast.literal_eval for Python-style dicts
    3. Call extract_actions_from_reasoning() to convert reasoning text
    4. Try simple pattern matching for common patterns
    5. Return original response unchanged if all else fails
    
    This defends against model outputs that are reasoning text instead of JSON.
    """
    if not isinstance(response, str):
        response = str(response)
    
    response = response.strip()
    
    # Stage 1: Fast path - check if already valid JSON with tool_calls
    if 'tool_calls' in response:
        try:
            data = json.loads(response)
            if isinstance(data, dict) and 'tool_calls' in data:
                return response  # Already valid
        except json.JSONDecodeError:
            pass
    
    # Stage 2: Try ast.literal_eval for Python-style dicts (single quotes)
    try:
        data = ast.literal_eval(response)
        if isinstance(data, dict) and 'tool_calls' in data:
            return json.dumps(data)
    except (ValueError, SyntaxError):
        pass
    
    # Stage 3: Call extract_actions_from_reasoning to convert reasoning text
    try:
        extracted = extract_actions_from_reasoning(response, task)
        if extracted:
            return extracted
    except Exception as e:
        logger.debug(f"extract_actions_from_reasoning failed: {e}")
    
    # Stage 4: Simple pattern matching for top-level values
    # If response looks like just an email, order_id, etc., wrap it
    email_match = re.search(r'[\w\.-]+@[\w\.-]+', response)
    if email_match and len(response) < 200:
        email = email_match.group(0)
        return json.dumps({
            'tool_calls': [{
                'name': 'find_user_id_by_email',
                'arguments': {'email': email}
            }]
        })
    
    # If response looks like just an order ID
    order_match = re.search(r'#W\d{4,}', response, re.I)
    if order_match and len(response) < 200:
        order_id = order_match.group(0)
        return json.dumps({
            'tool_calls': [{
                'name': 'get_order_details',
                'arguments': {'order_id': order_id}
            }]
        })
    
    # Stage 5: Return original if no normalization applies
    return response

print("✓ normalize_response_to_json function defined")

✓ normalize_response_to_json function defined


## Test Reward Function

Test the reward function on a sample task to verify it works correctly.

In [17]:
# Initialize reward function
reward_function = TauBenchRewardFunction(tool_executor)

# Test 1: Perfect match (should get high reward)
print("\n📋 Test 1: Perfect Response Match")
print("-" * 80)

test_task = rl_tasks[0]  # Use first real task
print(f"Task: {test_task.task_id}")
print(f"Expected actions: {[a.name for a in test_task.expected_actions]}")

# Construct perfect response
perfect_response = json.dumps({
    "tool_calls": [
        {"name": a.name, "arguments": a.arguments}
        for a in test_task.expected_actions
    ]
})

reward, metrics = reward_function.calculate_reward(perfect_response, test_task)
print(f"\n✅ Perfect Match Results:")
print(f"   Reward: {reward:.3f}")
print(f"   Correct actions: {metrics['correct_action_names']}/{metrics['total_expected']}")
print(f"   Correct arguments: {metrics['correct_arguments']}/{metrics['total_expected']}")
print(f"   Precision: {metrics['precision']:.2f}, Recall: {metrics['recall']:.2f}, F1: {metrics['f1']:.2f}")

if reward < 0.7:
    print(f"   ⚠️  WARNING: Expected reward > 0.7 for perfect match, got {reward:.3f}")
else:
    print(f"   ✅ PASS: Reward is appropriately high")

# Test 2: Correct actions, slightly different arguments
print("\n📋 Test 2: Correct Actions, Different Arguments")
print("-" * 80)

if len(test_task.expected_actions) >= 2:
    # Modify arguments slightly
    modified_response = json.dumps({
        "tool_calls": [
            {
                "name": a.name, 
                "arguments": {k: str(v) + "_modified" if isinstance(v, str) and '@' not in v else v 
                             for k, v in a.arguments.items()}
            }
            for a in test_task.expected_actions
        ]
    })
    
    reward2, metrics2 = reward_function.calculate_reward(modified_response, test_task)
    print(f"\n✅ Modified Args Results:")
    print(f"   Reward: {reward2:.3f}")
    print(f"   Correct actions: {metrics2['correct_action_names']}/{metrics2['total_expected']}")
    print(f"   Correct arguments: {metrics2['correct_arguments']}/{metrics2['total_expected']}")
    
    if reward2 > 0.3 and reward2 < reward:
        print(f"   ✅ PASS: Reward is positive but lower than perfect match")
    else:
        print(f"   ⚠️  WARNING: Expected 0.3 < reward < {reward:.3f}, got {reward2:.3f}")

# Test 3: Wrong actions
print("\n📋 Test 3: Wrong Actions")
print("-" * 80)

wrong_response = json.dumps({
    "tool_calls": [
        {"name": "transfer_to_human_agents", "arguments": {"reason": "test"}},
        {"name": "get_user_details", "arguments": {"user_id": "123"}}
    ]
})

reward3, metrics3 = reward_function.calculate_reward(wrong_response, test_task)
print(f"\n✅ Wrong Actions Results:")
print(f"   Reward: {reward3:.3f}")
print(f"   Correct actions: {metrics3['correct_action_names']}/{metrics3['total_expected']}")
print(f"   Incorrect actions: {metrics3['incorrect_actions']}")

if reward3 < 0.2:
    print(f"   ✅ PASS: Reward is appropriately low")
else:
    print(f"   ⚠️  WARNING: Expected reward < 0.2 for wrong actions, got {reward3:.3f}")

# Test 4: Empty response
print("\n📋 Test 4: Empty Response")
print("-" * 80)

empty_response = "{}"
reward4, metrics4 = reward_function.calculate_reward(empty_response, test_task)
print(f"\n✅ Empty Response Results:")
print(f"   Reward: {reward4:.3f}")
print(f"   Parsed actions: {metrics4['total_parsed']}")

if reward4 == 0.0:
    print(f"   ✅ PASS: Empty response gets zero reward")
else:
    print(f"   ⚠️  WARNING: Expected reward = 0.0, got {reward4:.3f}")

# Summary
print("\n" + "=" * 80)
print("VALIDATION SUMMARY")
print("=" * 80)

total_tests = 4
passed_tests = 0

if reward >= 0.7:
    passed_tests += 1
if reward2 > 0.3 and reward2 < reward:
    passed_tests += 1
if reward3 < 0.2:
    passed_tests += 1
if reward4 == 0.0:
    passed_tests += 1

print(f"Tests passed: {passed_tests}/{total_tests}")

if passed_tests == total_tests:
    print("✅ ALL TESTS PASSED! Reward function is working correctly.")
    print("\n🚀 Ready to start training!")

# Display statistics
print("\n📊 Reward Function Statistics:")
stats = reward_function.get_statistics()
for key, value in stats.items():
    if isinstance(value, float):
        print(f"   {key}: {value:.3f}")
    else:
        print(f"   {key}: {value}")


[__main__|INFO]Task task_1 - Reward: 5.000
[__main__|INFO]  Parsed: ['find_user_id_by_email', 'get_order_details', 'cancel_pending_order']
[__main__|INFO]  Expected: ['find_user_id_by_email', 'get_order_details', 'cancel_pending_order']
[__main__|INFO]  Metrics: correct_names=3/3, correct_args=3, completeness_penalty=0.00
[__main__|INFO]Task task_1 - Reward: 4.500
[__main__|INFO]  Parsed: ['find_user_id_by_email', 'get_order_details', 'cancel_pending_order']
[__main__|INFO]  Expected: ['find_user_id_by_email', 'get_order_details', 'cancel_pending_order']
[__main__|INFO]  Metrics: correct_names=3/3, correct_args=2, completeness_penalty=0.00
[__main__|INFO]Task task_1 - Reward: -1.000
[__main__|INFO]  Parsed: ['transfer_to_human_agents', 'get_user_details']
[__main__|INFO]  Expected: ['find_user_id_by_email', 'get_order_details', 'cancel_pending_order']
[__main__|INFO]  Metrics: correct_names=0/3, correct_args=0, completeness_penalty=-0.40
[__main__|WARNING]Skipping malformed tool call (


📋 Test 1: Perfect Response Match
--------------------------------------------------------------------------------
Task: task_1
Expected actions: ['find_user_id_by_email', 'get_order_details', 'cancel_pending_order']

✅ Perfect Match Results:
   Reward: 5.000
   Correct actions: 3/3
   Correct arguments: 3/3
   Precision: 1.00, Recall: 1.00, F1: 1.00
   ✅ PASS: Reward is appropriately high

📋 Test 2: Correct Actions, Different Arguments
--------------------------------------------------------------------------------

✅ Modified Args Results:
   Reward: 4.500
   Correct actions: 3/3
   Correct arguments: 2/3
   ✅ PASS: Reward is positive but lower than perfect match

📋 Test 3: Wrong Actions
--------------------------------------------------------------------------------

✅ Wrong Actions Results:
   Reward: -1.000
   Correct actions: 0/3
   Incorrect actions: 2
   ✅ PASS: Reward is appropriately low

📋 Test 4: Empty Response
---------------------------------------------------------------

## Data Splitting and Training Strategy

### Data Split Strategy
After filtering and validation, we split the tasks into three datasets:
- **Train (70%)**: Used for SFT and RL training
- **Validation (15%)**: Used for monitoring training progress
- **Test (15%)**: Used for final evaluation only (no information leakage)

### Training Phases

#### Phase 0: SFT (Supervised Fine-Tuning)
- **Dataset**: 100% of training split
- **Goal**: Teach JSON formatting, tool selection, and argument extraction
- **Duration**: 1 epoch with controlled learning rate
- **Key insight**: SFT benefits from full data diversity - uses all 70% of training split

#### Phase 1: RL (GRPO)
- **Dataset**: 100% of training split with reward function
- **Goal**: Optimize for task completion using learned reward signal
- **Strategy**: RL exploration is driven by rewards, not data scarcity
- **Key insight**: Even though RL sees the same tasks as SFT, it explores novel solutions via the reward signal

### Why This Approach Works
1. **SFT Phase** (supervised): More data = better generalization across task patterns
2. **RL Phase** (reinforcement): Exploration is driven by rewards, so "seeing data in SFT" doesn't limit RL innovation
3. **Data Efficiency**: No data is wasted; all training data contributes to both phases
4. **Evaluation Integrity**: Validation and test sets remain completely separate



In [18]:
def prepare_dataset_for_training(rl_tasks: List[RLTask], chatml_prompts: List[str] = None):
    """Prepare dataset for GRPO trainer - use ChatML prompts for consistency with model training."""
    from datasets import Dataset
    
    # Use ChatML prompts if provided, otherwise fall back to plain prompts
    prompts = chatml_prompts if chatml_prompts is not None else [t.prompt for t in rl_tasks]
    
    if chatml_prompts and len(chatml_prompts) != len(rl_tasks):
        logger.warning(f"ChatML prompts count {len(chatml_prompts)} != tasks {len(rl_tasks)}, using plain prompts")
        prompts = [t.prompt for t in rl_tasks]
    
    data_dict = {
        'prompt': prompts,  # Use ChatML prompts for RL training
        'task_id': [t.task_id for t in rl_tasks],
        'expected_actions': [t.gold_actions_str for t in rl_tasks],
        'metadata': [t.metadata for t in rl_tasks]
    }
    
    # Convert to HuggingFace Dataset
    dataset = Dataset.from_dict(data_dict)
    return dataset

# Will populate chatml_prompts later after preparing them
train_dataset = None  # Will be set after ChatML conversion

print("✓ prepare_dataset_for_training defined (awaiting ChatML conversion)")


✓ prepare_dataset_for_training defined (awaiting ChatML conversion)


In [19]:
import random
from math import floor

def split_tasks(tasks: List[RLTask], train_ratio: float = 0.7, val_ratio: float = 0.15, seed: int = 3407) -> tuple:
    """Split tasks into train/val/test sets with optional random seed.
    
    Args:
        tasks: List of RLTask objects to split
        train_ratio: Fraction for training set (default 0.7)
        val_ratio: Fraction for validation set (default 0.15)
        seed: Random seed for reproducibility
    
    Returns:
        Tuple of (train_tasks, val_tasks, test_tasks)
    """
    if seed is not None:
        random.seed(seed)
    
    n = len(tasks)
    idxs = list(range(n))
    random.shuffle(idxs)
    
    n_train = floor(n * train_ratio)
    n_val = floor(n * val_ratio)
    train_idxs = idxs[:n_train]
    val_idxs = idxs[n_train:n_train+n_val]
    test_idxs = idxs[n_train+n_val:]
    
    def take(indices):
        return [tasks[i] for i in indices]
    
    return take(train_idxs), take(val_idxs), take(test_idxs)

# Split tasks into train/val/test
train_tasks, val_tasks, test_tasks = split_tasks(rl_tasks, 0.7, 0.15, seed=3407)
print("=" * 80)
print("DATA SPLIT RESULTS")
print("=" * 80)
print(f"✓ Total filtered tasks:  {len(rl_tasks)}")
print(f"  ├─ Train (70%):        {len(train_tasks)} tasks")
print(f"  ├─ Validation (15%):   {len(val_tasks)} tasks")
print(f"  └─ Test (15%):         {len(test_tasks)} tasks")
print(f"\n✓ Train split size: {len(train_tasks)} tasks (before SFT/RL split)")
print(f"✓ Validation split: {len(val_tasks)} tasks")
print(f"✓ Test split:       {len(test_tasks)} tasks")
print("=" * 80)

# Allocate 30% of train split for SFT, 70% for RL
sft_ratio = 0.60
sft_size = max(1, int(len(train_tasks) * sft_ratio))
sft_tasks = random.sample(train_tasks, sft_size)
rl_train_tasks = train_tasks

print("SFT/RL allocation within train split:")
print(f"  ├─ SFT tasks: {len(sft_tasks)} ({sft_ratio*100:.0f}% of train split)")
print(f"  └─ RL tasks:  {len(rl_train_tasks)} ({100 - sft_ratio*100:.0f}% of train split)")

DATA SPLIT RESULTS
✓ Total filtered tasks:  1323
  ├─ Train (70%):        926 tasks
  ├─ Validation (15%):   198 tasks
  └─ Test (15%):         199 tasks

✓ Train split size: 926 tasks (before SFT/RL split)
✓ Validation split: 198 tasks
✓ Test split:       199 tasks
SFT/RL allocation within train split:
  ├─ SFT tasks: 555 (60% of train split)
  └─ RL tasks:  926 (40% of train split)


In [20]:
# Convert RL prompts to ChatML format (system + user) for consistent tokenization
def convert_prompts_to_chatml(rl_tasks: List[RLTask], tokenizer, system_prompt: str):
    """Convert plain prompts to ChatML messages and format with tokenizer."""
    chatml_prompts = []
    for t in rl_tasks:
        # Extract user query from the plain prompt
        # Format is typically "SYSTEM_PROMPT\n\nQuery: <user_query>"
        user_query = t.prompt.split("Query:", 1)[-1].strip() if "Query:" in t.prompt else t.prompt
        
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_query},
        ]
        # Apply ChatML template and request generation prompt
        chatml_text = tokenizer.apply_chat_template(
            messages, 
            tokenize=False, 
            add_generation_prompt=True
        )
        chatml_prompts.append(chatml_text)
    return chatml_prompts


# Convert RL (70%) train split prompts to ChatML
rl_chatml_prompts = convert_prompts_to_chatml(rl_train_tasks, tokenizer, SYSTEM_PROMPT)

# NOW build training dataset with ChatML prompts from RL portion of train split
train_dataset = prepare_dataset_for_training(rl_train_tasks, chatml_prompts=rl_chatml_prompts)

# Verify conversion
print(f"✓ Converted {len(rl_chatml_prompts)} RL TRAIN prompts to ChatML format")
print(f"\nExample ChatML prompt (first 300 chars):")
print(rl_chatml_prompts[0][:300])

# Validate dataset was created correctly
assert len(train_dataset) == len(rl_train_tasks), f"Dataset size mismatch: {len(train_dataset)} != {len(rl_train_tasks)}"
assert 'prompt' in train_dataset.column_names, "Missing 'prompt' column in dataset"

print(f"\n✓ Prepared {len(train_dataset)} training examples (ChatML prompts from RL train split)")
print(f"✓ Dataset columns: {train_dataset.column_names}")
print(f"\nExample:")
print(f"  Task ID: {train_dataset[0]['task_id']}")
print(f"  Prompt length: {len(train_dataset[0]['prompt'])} chars")
print(f"  Has expected actions: {'Yes' if train_dataset[0]['expected_actions'] else 'No'}")

✓ Converted 926 RL TRAIN prompts to ChatML format

Example ChatML prompt (first 300 chars):
<|im_start|>system
You are a customer service API. Output ONLY valid JSON.

CRITICAL RULES:
1. Output format: {"tool_calls": [{"name": "tool_name", "arguments": {...}}...]}
2. NEVER output plain text, explanations, or reasoning
3. Use only provided tools and actual data from customer request
4. Use 

✓ Prepared 926 training examples (ChatML prompts from RL train split)
✓ Dataset columns: ['prompt', 'task_id', 'expected_actions', 'metadata']

Example:
  Task ID: task_1997
  Prompt length: 2219 chars
  Has expected actions: Yes


## Prepare Validation Dataset

Prepare validation set for periodic evaluation during training.

In [21]:
# Convert validation split prompts to ChatML format
chatml_val_prompts = convert_prompts_to_chatml(val_tasks, tokenizer, SYSTEM_PROMPT)

# Build validation dataset with ChatML prompts
val_dataset = prepare_dataset_for_training(val_tasks, chatml_prompts=chatml_val_prompts)

# Build validation prompt→task cache for reward calculation
VAL_TASK_ID_TO_CHATML = {val_tasks[i].task_id: chatml_val_prompts[i] for i in range(len(val_tasks))}

# Also prepare test split for final evaluation (after training complete)
chatml_test_prompts = convert_prompts_to_chatml(test_tasks, tokenizer, SYSTEM_PROMPT)
TEST_TASK_ID_TO_CHATML = {test_tasks[i].task_id: chatml_test_prompts[i] for i in range(len(test_tasks))}

print(f"✓ Prepared {len(val_dataset)} validation examples (ChatML prompts from VAL split)")
print(f"✓ Prepared {len(test_tasks)} test examples (ChatML prompts from TEST split)")
print(f"✓ Validation will be used for periodic checks during training")
print(f"✓ Test split will be used for final evaluation only")

✓ Prepared 198 validation examples (ChatML prompts from VAL split)
✓ Prepared 199 test examples (ChatML prompts from TEST split)
✓ Validation will be used for periodic checks during training
✓ Test split will be used for final evaluation only


## Phase 0: SFT Pretraining for JSON Tool-Calls

Before RL (GRPO), run a short supervised fine-tuning (SFT) pass to teach strict JSON `tool_calls` formatting and tool sequencing on gold actions. This reduces parse failures and stabilizes RL rewards. We convert each task into ChatML messages: system + user (query) → assistant (gold `tool_calls`).

In [22]:
# Phase 0: SFT Pretraining for JSON Tool-Calls
# Prepare model for training with FastLanguageModel helper

# CRITICAL: Prepare 4-bit LoRA model for training (Unsloth requirement)
# This enables gradient checkpointing, disables cache, and optimizes VRAM usage
from unsloth import FastLanguageModel
model = FastLanguageModel.for_training(model)

print("✓ Model prepared for training (gradient checkpointing enabled, cache disabled)")

# Build ChatML SFT dataset
from datasets import Dataset
from trl import SFTTrainer, SFTConfig
from transformers import DataCollatorForLanguageModeling
import json
from unsloth import is_bfloat16_supported


def build_sft_dataset(rl_tasks: List[RLTask]):
    texts = []
    for t in rl_tasks:
        # Extract user query from the plain prompt (SYSTEM_PROMPT + "Query: ...")
        user_query = t.prompt.split("Query:", 1)[-1].strip() if "Query:" in t.prompt else t.prompt
        # Wrap gold actions into the required top-level container
        try:
            gold_list = json.loads(t.gold_actions_str) if t.gold_actions_str else []
        except Exception:
            gold_list = []
        assistant_json = {"tool_calls": gold_list}
        assistant_text = json.dumps(assistant_json, separators=(",", ":"))  # compact JSON
        messages = [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": user_query},
            {"role": "assistant", "content": assistant_text},
        ]
        # Produce ChatML-formatted text
        text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
        texts.append(text)
    return Dataset.from_dict({"text": texts})


sft_dataset = build_sft_dataset(sft_tasks)
print(f"✓ SFT dataset built: {len(sft_dataset)} examples")
output_dir = "outputs/tau_bench_rl"

sft_config = SFTConfig(
    output_dir=output_dir + "/sft",
    num_train_epochs=2,  # short pass to learn formatting
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    learning_rate=1e-4,
    warmup_steps=50,
    bf16=is_bfloat16_supported(),
    fp16=not is_bfloat16_supported(),
    optim="adamw_8bit",
    logging_steps=10,
    save_steps=200,
    save_total_limit=2,
)

sft_trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    args=sft_config,
    train_dataset=sft_dataset,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

print("\n🚀 Starting SFT pretraining (JSON tool-calls)...")
sft_trainer.train()
print("✓ SFT pretraining complete. Proceeding to RL (GRPO).")

✓ Model prepared for training (gradient checkpointing enabled, cache disabled)
✓ SFT dataset built: 555 examples


Unsloth: Tokenizing ["text"] (num_proc=24): 100%|██████████████████████████████████████████████| 555/555 [00:07<00:00, 77.89 examples/s]



🚀 Starting SFT pretraining (JSON tool-calls)...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 555 | Num Epochs = 2 | Total steps = 70
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 7,962,624 of 20,922,719,808 (0.04% trained)
[unsloth_zoo.log|INFO]Unsloth: num_items_in_batch = 10119
Unsloth: Compiling kernels: 0it [00:00, ?it/s]


Unsloth: Will smartly offload gradients to save VRAM!


Unsloth: Compiling kernels: 100%|████████████████| 1/1 [00:00<00:00,  4.74it/s, triton_poi_fused__to_copy_add_clamp_mul_sigmoid_slice_0]
Unsloth: Compiling kernels: 0it [00:00, ?it/s]
Unsloth: Compiling kernels: 100%|█████████████████| 1/1 [00:00<00:00,  5.11it/s, triton_red_fused__to_copy_add_div_expand_mul_pow_sum_0]
[unsloth_zoo.log|INFO]Fused CE Loss [bsz=2][qlen=672][vocab_size=201088][n_chunks=1]
Unsloth: Compiling kernels: 100%|██████████████████████████████████████████| 2/2 [00:00<00:00,  3.04it/s, triton_poi_fused_zeros_like_1]
Unsloth: Compiling kernels: 100%|█| 2/2 [00:00<00:00,  2.53it/s, triton_poi_fused__softmax__softmax_backward_data__to_copy_exp_scatter_s
Unsloth: Compiling kernels: 100%|████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  4.78it/s, cpp_fused_0]
Unsloth: Compiling kernels: 0it [00:00, ?it/s]
Unsloth: Compiling kernels: 100%|████████████████| 2/2 [00:00<00:00, 13.27it/s, triton_poi_fused__to_copy_add_clamp_mul_sigmoid_slice_1]

Step,Training Loss
10,3.381300
20,2.237800
30,1.491000
40,1.084700
50,0.639900
60,0.432000
70,0.367400


[unsloth_zoo.log|INFO]Unsloth: num_items_in_batch = 9982
[unsloth_zoo.log|INFO]Fused CE Loss [bsz=2][qlen=638][vocab_size=201088][n_chunks=1]
[unsloth_zoo.log|INFO]Fused CE Loss [bsz=2][qlen=661][vocab_size=201088][n_chunks=1]
[unsloth_zoo.log|INFO]Fused CE Loss [bsz=2][qlen=637][vocab_size=201088][n_chunks=1]
[unsloth_zoo.log|INFO]Fused CE Loss [bsz=2][qlen=627][vocab_size=201088][n_chunks=1]
[unsloth_zoo.log|INFO]Fused CE Loss [bsz=2][qlen=695][vocab_size=201088][n_chunks=1]
[unsloth_zoo.log|INFO]Fused CE Loss [bsz=2][qlen=623][vocab_size=201088][n_chunks=1]
[unsloth_zoo.log|INFO]Fused CE Loss [bsz=2][qlen=622][vocab_size=201088][n_chunks=1]
[unsloth_zoo.log|INFO]Fused CE Loss [bsz=2][qlen=616][vocab_size=201088][n_chunks=1]
[unsloth_zoo.log|INFO]Unsloth: num_items_in_batch = 9751
[unsloth_zoo.log|INFO]Fused CE Loss [bsz=2][qlen=624][vocab_size=201088][n_chunks=1]
[unsloth_zoo.log|INFO]Fused CE Loss [bsz=2][qlen=628][vocab_size=201088][n_chunks=1]
[unsloth_zoo.log|INFO]Fused CE Loss 

✓ SFT pretraining complete. Proceeding to RL (GRPO).


## Validation Callback

Custom callback to run validation on val_tasks during RL training.

In [23]:
# Build prompt→task cache using ChatML prompts for fast lookup and strict JSON validator
# Map both original and ChatML prompts to tasks for compatibility
PROMPT_TO_TASK = {t.prompt: t for t in rl_train_tasks}
PROMPT_TO_TASK_STRIPPED = {t.prompt.strip(): t for t in rl_train_tasks}
# Map ChatML prompts (for RL training) to tasks - built from RL train split only
PROMPT_TO_TASK_CHATML = {rl_chatml_prompts[i]: rl_train_tasks[i] for i in range(len(rl_train_tasks))}
TASK_ID_TO_CHATML = {rl_train_tasks[i].task_id: rl_chatml_prompts[i] for i in range(len(rl_train_tasks))}

def strict_validate_json_response(s: str) -> bool:
    try:
        obj = json.loads(s)
    except Exception:
        return False
    if not isinstance(obj, dict):
        return False
    calls = obj.get("tool_calls")
    if not isinstance(calls, list) or len(calls) == 0:
        return False
    for c in calls:
        if not isinstance(c, dict):
            return False
        name = c.get("name")
        args = c.get("arguments")
        if not isinstance(name, str) or not name.strip():
            return False
        if not isinstance(args, dict):
            return False
    return True

print("✓ Prompt→task cache built (original + ChatML) for RL split")

✓ Prompt→task cache built (original + ChatML) for RL split


In [24]:
from transformers import TrainerCallback
import time

class ValidationCallback(TrainerCallback):
    """Custom callback to evaluate model on validation set during RL training.
    
    Triggers on save events (every save_steps) since eval_strategy="no".
    """
    
    def __init__(self, model, tokenizer, val_tasks, reward_function, num_samples=10, eval_steps=100):
        self.model = model
        self.tokenizer = tokenizer
        self.val_tasks = val_tasks
        self.reward_function = reward_function
        self.num_samples = num_samples
        self.eval_steps = eval_steps
        self.validation_history = []
        self.last_eval_step = -1
    
    def on_save(self, args, state, control, **kwargs):
        """Run validation evaluation when checkpoint is saved.
        
        Since eval_strategy="no", we use on_save hook (triggered at save_steps).
        """
        # Prevent duplicate evaluation at the same step
        if state.global_step == self.last_eval_step:
            return control
        
        self.last_eval_step = state.global_step
        
        print(f"\n{'='*60}")
        print(f"🔍 Running validation at step {state.global_step}")
        print(f"{'='*60}")
        
        start_time = time.time()
        
        # Sample validation tasks
        eval_tasks = random.sample(self.val_tasks, min(self.num_samples, len(self.val_tasks)))
        
        total_reward = 0.0
        valid_responses = 0
        parse_failures = 0
        
        self.model.eval()
        
        for task in eval_tasks:
            # Use ChatML prompt (consistent with training)
            chatml_prompt = VAL_TASK_ID_TO_CHATML.get(task.task_id)
            if not chatml_prompt:
                user_query = task.prompt.split("Query:", 1)[-1].strip() if "Query:" in task.prompt else task.prompt
                messages = [
                    {"role": "system", "content": SYSTEM_PROMPT},
                    {"role": "user", "content": user_query},
                ]
                chatml_prompt = self.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
            
            inputs = self.tokenizer(chatml_prompt, return_tensors="pt").to(self.model.device)
            input_length = inputs['input_ids'].shape[1]
            
            with torch.no_grad():
                outputs = self.model.generate(
                    **inputs,
                    max_new_tokens=256,
                    temperature=0.2,
                    top_p=0.9,
                    do_sample=False,
                    pad_token_id=self.tokenizer.eos_token_id,
                )
            
            generated_tokens = outputs[0][input_length:]
            response = self.tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()
            
            if not response:
                continue
            
            # Normalize and validate
            norm = normalize_response_to_json(response, task)
            if not strict_validate_json_response(norm):
                parse_failures += 1
                total_reward += -0.5
            else:
                reward, metrics = self.reward_function.calculate_reward(norm, task)
                total_reward += reward
                valid_responses += 1
        
        self.model.train()
        
        avg_reward = total_reward / len(eval_tasks) if eval_tasks else 0.0
        success_rate = valid_responses / len(eval_tasks) if eval_tasks else 0.0
        
        elapsed = time.time() - start_time
        
        self.validation_history.append({
            'step': state.global_step,
            'avg_reward': avg_reward,
            'success_rate': success_rate,
            'parse_failures': parse_failures
        })
        
        print(f"\n📊 Validation Results:")
        print(f"  ├─ Average Reward:  {avg_reward:.3f}")
        print(f"  ├─ Success Rate:    {success_rate:.1%}")
        print(f"  ├─ Parse Failures:  {parse_failures}/{len(eval_tasks)}")
        print(f"  └─ Time:            {elapsed:.1f}s")
        print(f"{'='*60}\n")
        
        # Log to TensorBoard (if available)
        if hasattr(state, 'log_history'):
            try:
                # These will be logged to TensorBoard
                kwargs.get('metrics', {}).update({
                    'eval_reward': avg_reward,
                    'eval_success_rate': success_rate,
                    'eval_parse_failures': parse_failures,
                })
            except:
                pass
        
        return control

# Initialize validation callback
validation_callback = ValidationCallback(
    model=model,
    tokenizer=tokenizer,
    val_tasks=val_tasks,
    reward_function=reward_function,
    num_samples=10,  # Evaluate on 10 random validation tasks
    eval_steps=100   # Matches save_steps
)

print("✓ Validation callback initialized")
print(f"  → Will evaluate on {10} random val_tasks at each checkpoint save")
print(f"  → Checkpoints saved every 100 steps (matches save_steps config)")

✓ Validation callback initialized
  → Will evaluate on 10 random val_tasks at each checkpoint save
  → Checkpoints saved every 100 steps (matches save_steps config)


## Configure GRPO Training

Set up training configuration for Group Relative Policy Optimization.

**Training Parameters:**
- `num_train_epochs`: Number of training epochs
- `per_device_train_batch_size`: Batch size per GPU
- `gradient_accumulation_steps`: Accumulation for larger effective batch
- `learning_rate`: Learning rate for optimizer
- `bf16/fp16`: Mixed precision training

In [25]:
from transformers import TrainingArguments
from trl import GRPOConfig, GRPOTrainer
from unsloth import is_bfloat16_supported

# Training configuration
output_dir = "outputs/tau_bench_rl"
num_epochs = 2
batch_size = 2
gradient_accumulation_steps = 16
learning_rate = 5e-5

training_args = GRPOConfig(
    output_dir=output_dir,
    num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    learning_rate=learning_rate,
    warmup_steps=200,
    logging_steps=10,
    save_steps=100,
    save_total_limit=3,
    max_grad_norm=0.5,
    eval_strategy="no",  # Disable built-in evaluation (use custom callback)
    bf16=is_bfloat16_supported(),
    fp16=not is_bfloat16_supported(),
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="cosine",
    seed=3407,
    report_to="tensorboard",
    # Tighter, cheaper generation during RL with ChatML-formatted prompts
    max_completion_length=512,
    temperature=0.7,
    top_p=0.9,
)

print(f"✓ Training configuration:")
print(f"  Epochs: {num_epochs}")
print(f"  Batch size: {batch_size}")
print(f"  Gradient accumulation: {gradient_accumulation_steps}")
print(f"  Effective batch size: {batch_size * gradient_accumulation_steps}")
print(f"  Learning rate: {learning_rate}")
print(f"  Precision: {'BF16' if is_bfloat16_supported() else 'FP16'}")
print(f"  Validation: Custom callback (every checkpoint save, ~100 steps)")
print(f"  ✓ Using custom ValidationCallback for periodic checks on val_tasks")

✓ Training configuration:
  Epochs: 2
  Batch size: 2
  Gradient accumulation: 16
  Effective batch size: 32
  Learning rate: 5e-05
  Precision: BF16
  Validation: Custom callback (every checkpoint save, ~100 steps)
  ✓ Using custom ValidationCallback for periodic checks on val_tasks


In [26]:
## Initialize Comprehensive Metrics Tracker
"""
Tracks key training metrics during RL phase:
- Reward trajectory (mean, std, min, max per epoch)
- Parse failure rate trends
- Zero-reward rate (examples getting penalty)
- Action correctness breakdown (tool vs args)
- Execution success rate
"""

import collections
from typing import Dict, Any, List

class TrainingMetricsTracker:
    """Comprehensive metrics tracking during GRPO training."""
    
    def __init__(self):
        # Reward trajectory
        self.step_rewards = collections.defaultdict(list)  # step -> [rewards]
        self.epoch_rewards = collections.defaultdict(list)  # epoch -> [rewards]
        
        # Parse failures
        self.step_parse_failures = collections.defaultdict(int)  # step -> count
        self.step_total_examples = collections.defaultdict(int)   # step -> count
        
        # Zero-reward tracking
        self.step_zero_rewards = collections.defaultdict(int)  # step -> count
        
        # Action correctness breakdown
        self.step_tool_correct = collections.defaultdict(int)
        self.step_tool_total = collections.defaultdict(int)
        self.step_args_correct = collections.defaultdict(int)
        self.step_args_total = collections.defaultdict(int)
        
        # Execution success
        self.step_exec_success = collections.defaultdict(int)
        self.step_exec_total = collections.defaultdict(int)
        
        # Epoch-level summaries
        self.epoch_summaries = []
        self.current_epoch = 0
        
    def log_reward_batch(self, step: int, epoch: int, rewards: List[float]):
        """Log rewards for a batch."""
        self.step_rewards[step].extend(rewards)
        self.epoch_rewards[epoch].extend(rewards)
        self.step_total_examples[step] += len(rewards)
        
        # Track zero rewards
        zero_count = sum(1 for r in rewards if r <= 0.0)
        self.step_zero_rewards[step] += zero_count
        
    def log_parsing_results(self, step: int, parse_failures: int, total: int):
        """Log parsing success/failure rates."""
        self.step_parse_failures[step] += parse_failures
        self.step_total_examples[step] += total
        
    def log_action_correctness(self, step: int, epoch: int, metrics: Dict[str, Any]):
        """Log action correctness breakdown from reward function metrics."""
        # Tool correctness
        if 'correct_action_names' in metrics and 'total_expected' in metrics:
            self.step_tool_correct[step] += metrics['correct_action_names']
            self.step_tool_total[step] += metrics['total_expected']
            
            # Also track per epoch
            if epoch not in self.epoch_summaries:
                self.epoch_summaries.append({'epoch': epoch})
        
        # Arguments correctness  
        if 'correct_arguments' in metrics and 'total_expected' in metrics:
            self.step_args_correct[step] += metrics['correct_arguments']
            self.step_args_total[step] += metrics['total_expected']
    
    def log_execution_success(self, step: int, success: bool, total: int = 1):
        """Log tool execution success rate."""
        if success:
            self.step_exec_success[step] += 1
        self.step_exec_total[step] += total
        
    def get_step_summary(self, step: int) -> Dict[str, Any]:
        """Get metrics summary for a specific step."""
        rewards = self.step_rewards.get(step, [])
        parse_failures = self.step_parse_failures.get(step, 0)
        zero_rewards = self.step_zero_rewards.get(step, 0)
        total_examples = self.step_total_examples.get(step, 1)
        
        tool_correct = self.step_tool_correct.get(step, 0)
        tool_total = self.step_tool_total.get(step, 1)
        args_correct = self.step_args_correct.get(step, 0)
        args_total = self.step_args_total.get(step, 1)
        
        exec_success = self.step_exec_success.get(step, 0)
        exec_total = self.step_exec_total.get(step, 1)
        
        return {
            'step': step,
            'reward_mean': float(sum(rewards) / len(rewards)) if rewards else 0.0,
            'reward_std': float((sum((r - (sum(rewards)/len(rewards)))**2 for r in rewards) / len(rewards))**0.5) if len(rewards) > 1 else 0.0,
            'reward_min': float(min(rewards)) if rewards else 0.0,
            'reward_max': float(max(rewards)) if rewards else 0.0,
            'parse_failure_rate': parse_failures / max(total_examples, 1),
            'zero_reward_rate': zero_rewards / max(total_examples, 1),
            'tool_correctness': tool_correct / max(tool_total, 1),
            'args_correctness': args_correct / max(args_total, 1),
            'execution_success_rate': exec_success / max(exec_total, 1),
        }
    
    def get_epoch_summary(self, epoch: int) -> Dict[str, Any]:
        """Get aggregated metrics for an epoch."""
        rewards = self.epoch_rewards.get(epoch, [])
        
        if not rewards:
            return {'epoch': epoch, 'num_examples': 0}
        
        return {
            'epoch': epoch,
            'num_examples': len(rewards),
            'reward_mean': float(sum(rewards) / len(rewards)),
            'reward_std': float((sum((r - (sum(rewards)/len(rewards)))**2 for r in rewards) / len(rewards))**0.5) if len(rewards) > 1 else 0.0,
            'reward_min': float(min(rewards)),
            'reward_max': float(max(rewards)),
            'reward_median': float(sorted(rewards)[len(rewards)//2]) if rewards else 0.0,
            'positive_reward_rate': sum(1 for r in rewards if r > 0.0) / len(rewards),
        }
    
    def print_step_report(self, step: int):
        """Print formatted report for a step."""
        summary = self.get_step_summary(step)
        print(f"\n📊 Step {step} Metrics:")
        print(f"  Reward Trajectory:")
        print(f"    ├─ Mean: {summary['reward_mean']:.4f} ± {summary['reward_std']:.4f}")
        print(f"    ├─ Range: [{summary['reward_min']:.4f}, {summary['reward_max']:.4f}]")
        print(f"  Parse Failure Rate: {summary['parse_failure_rate']:.2%}")
        print(f"  Zero-Reward Rate: {summary['zero_reward_rate']:.2%}")
        print(f"  Action Correctness:")
        print(f"    ├─ Tool Correct: {summary['tool_correctness']:.2%}")
        print(f"    └─ Args Correct: {summary['args_correctness']:.2%}")
        print(f"  Execution Success: {summary['execution_success_rate']:.2%}")
    
    def print_epoch_report(self, epoch: int):
        """Print formatted report for an epoch."""
        summary = self.get_epoch_summary(epoch)
        if summary['num_examples'] == 0:
            print(f"\n📊 Epoch {epoch}: No data")
            return
            
        print(f"\n📊 Epoch {epoch} Summary:")
        print(f"  Examples: {summary['num_examples']}")
        print(f"  Reward Trajectory:")
        print(f"    ├─ Mean: {summary['reward_mean']:.4f} ± {summary['reward_std']:.4f}")
        print(f"    ├─ Range: [{summary['reward_min']:.4f}, {summary['reward_max']:.4f}]")
        print(f"    ├─ Median: {summary['reward_median']:.4f}")
        print(f"    └─ Positive Rate: {summary['positive_reward_rate']:.2%}")
    
    def print_training_report(self):
        """Print comprehensive training report."""
        print("\n" + "="*80)
        print("COMPREHENSIVE TRAINING METRICS REPORT")
        print("="*80)
        
        # Per-epoch summaries
        epochs = sorted(set(self.epoch_rewards.keys()))
        if epochs:
            print("\n📈 Epoch Summaries:")
            for epoch in epochs:
                self.print_epoch_report(epoch)
        
        # Overall statistics
        all_rewards = [r for rewards in self.epoch_rewards.values() for r in rewards]
        if all_rewards:
            print(f"\n📊 Overall Training Statistics:")
            print(f"  Total Examples Trained: {len(all_rewards)}")
            print(f"  Reward Mean: {sum(all_rewards)/len(all_rewards):.4f}")
            print(f"  Reward Range: [{min(all_rewards):.4f}, {max(all_rewards):.4f}]")
            print(f"  Positive Rewards: {sum(1 for r in all_rewards if r > 0.0)} ({sum(1 for r in all_rewards if r > 0.0)/len(all_rewards):.2%})")
            print(f"  Zero/Negative Rewards: {sum(1 for r in all_rewards if r <= 0.0)} ({sum(1 for r in all_rewards if r <= 0.0)/len(all_rewards):.2%})")

# Initialize tracker
metrics_tracker = TrainingMetricsTracker()
print("✅ Comprehensive metrics tracker initialized")
print("   Tracking: rewards, parse failures, zero-reward rate, action correctness, execution success")

✅ Comprehensive metrics tracker initialized
   Tracking: rewards, parse failures, zero-reward rate, action correctness, execution success


## Create Reward Function Wrapper

Wrap the reward function for compatibility with GRPO trainer.

In [27]:
def reward_function_wrapper(completions, **kwargs) -> List[float]:
    """Wrapper for reward function compatible with GRPO trainer.
    
    Steps:
    - Map each completion back to its task using cached prompt→task maps
    - Try ChatML cache first, then fallback to original prompts
    - Normalize response to strict JSON with tool_calls
    - Soft penalty for parse failures (not zero - let RL explore)
    - Delegate to reward_function for detailed scoring
    - Log comprehensive metrics for training analysis
    - Return raw rewards (GRPOTrainer handles advantage computation internally)
    """
    rewards = []
    prompts = kwargs.get('prompts', None)
    current_step = kwargs.get('step', 0)
    current_epoch = kwargs.get('epoch', 0)
    
    parse_failures = 0
    
    for idx, completion in enumerate(completions):
        # Extract response text from completion
        if isinstance(completion, list) and len(completion) > 0:
            response = completion[0].get("content", "")
        elif isinstance(completion, dict):
            response = completion.get("content", "")
        else:
            response = str(completion)
        
        # Resolve task via cached prompt maps (try ChatML first, then original)
        matching_task = None
        if prompts and idx < len(prompts):
            prompt = prompts[idx]
            # Try ChatML cache first (since RL uses ChatML prompts)
            matching_task = PROMPT_TO_TASK_CHATML.get(prompt)
            # Fallback to original prompt cache
            if matching_task is None:
                matching_task = PROMPT_TO_TASK.get(prompt)
            if matching_task is None:
                matching_task = PROMPT_TO_TASK_STRIPPED.get(prompt.strip())
        
        if matching_task is None:
            logger.warning(f"No matching task found for completion {idx}")
            rewards.append(-0.5)  # Soft penalty
            parse_failures += 1
            continue
        
        # Normalize response first
        normalized_response = normalize_response_to_json(response, matching_task)
        
        # Soft penalty for parse failures (NOT zero - let RL explore)
        if not strict_validate_json_response(normalized_response):
            rewards.append(-0.5)  # Soft negative, not death sentence
            parse_failures += 1
            continue
        
        # Score
        try:
            reward, metrics = reward_function.calculate_reward(normalized_response, matching_task)
            rewards.append(reward)
            
            # Log action correctness breakdown
            if hasattr(metrics_tracker, 'log_action_correctness'):
                metrics_tracker.log_action_correctness(current_step, current_epoch, metrics)
        except Exception as e:
            logger.error(f"Error calculating reward: {e}")
            rewards.append(-0.5)  # Soft penalty for errors too
            parse_failures += 1
    
    # Log batch-level metrics
    if hasattr(metrics_tracker, 'log_reward_batch'):
        metrics_tracker.log_reward_batch(current_step, current_epoch, rewards)
    if hasattr(metrics_tracker, 'log_parsing_results'):
        metrics_tracker.log_parsing_results(current_step, parse_failures, len(completions))
    
    # GRPOTrainer handles advantage computation internally - just return raw rewards
    return rewards

print("✓ Reward wrapper: raw rewards + soft penalties for parse failures")


✓ Reward wrapper: raw rewards + soft penalties for parse failures


In [28]:
## Enhanced Reward Wrapper with Metrics Tracking

def reward_function_wrapper_with_metrics(completions, **kwargs) -> List[float]:
    """Enhanced reward wrapper that logs comprehensive training metrics."""
    rewards = []
    prompts = kwargs.get('prompts', None)
    
    # Try to extract step/epoch from kwargs
    current_step = kwargs.get('step', kwargs.get('global_step', 0))
    current_epoch = kwargs.get('epoch', 0)
    
    parse_failures = 0
    
    for idx, completion in enumerate(completions):
        # Extract response text from completion
        if isinstance(completion, list) and len(completion) > 0:
            response = completion[0].get("content", "")
        elif isinstance(completion, dict):
            response = completion.get("content", "")
        else:
            response = str(completion)
        
        # Resolve task via cached prompt maps (try ChatML first, then original)
        matching_task = None
        if prompts and idx < len(prompts):
            prompt = prompts[idx]
            matching_task = PROMPT_TO_TASK_CHATML.get(prompt)
            if matching_task is None:
                matching_task = PROMPT_TO_TASK.get(prompt)
            if matching_task is None:
                matching_task = PROMPT_TO_TASK_STRIPPED.get(prompt.strip())
        
        if matching_task is None:
            rewards.append(-0.5)
            parse_failures += 1
            continue
        
        # Normalize response first
        normalized_response = normalize_response_to_json(response, matching_task)
        
        # Soft penalty for parse failures
        if not strict_validate_json_response(normalized_response):
            rewards.append(-0.5)
            parse_failures += 1
            continue
        
        # Score with detailed metrics
        try:
            reward, metrics = reward_function.calculate_reward(normalized_response, matching_task)
            rewards.append(reward)
            
            # Log action correctness breakdown
            metrics_tracker.log_action_correctness(current_step, current_epoch, metrics)
        except Exception as e:
            rewards.append(-0.5)
            parse_failures += 1
    
    # Log batch-level metrics to tracker
    metrics_tracker.log_reward_batch(current_step, current_epoch, rewards)
    metrics_tracker.log_parsing_results(current_step, parse_failures, len(completions))
    
    return rewards

print("✓ Enhanced reward wrapper with metrics tracking enabled")

✓ Enhanced reward wrapper with metrics tracking enabled


## Initialize GRPO Trainer

Create the trainer with model, config, and reward function.

In [29]:
trainer = GRPOTrainer(
    model=model,
    tokenizer=tokenizer,
    reward_funcs=[reward_function_wrapper_with_metrics],  # Enhanced wrapper with metrics tracking
    args=training_args,
    train_dataset=train_dataset,
    # eval_dataset removed: using custom ValidationCallback instead
    callbacks=[validation_callback],  # Custom validation during training
)

print("✓ GRPO Trainer initialized (with custom validation callback)")
print(f"  ├─ Training on {len(train_dataset)} examples")
print(f"  └─ Validating on {len(val_dataset)} examples via custom callback")


✓ GRPO Trainer initialized (with custom validation callback)
  ├─ Training on 926 examples
  └─ Validating on 198 examples via custom callback


## Start Training

⚠️ **Warning**: This will take significant time depending on your hardware!

In [30]:
print("🚀 Starting RL training...")
print("This may take a while. Monitor progress in TensorBoard.")
print(f"\nTo view TensorBoard: tensorboard --logdir {output_dir}")

trainer.train()

# After training, report parse failure stats
stats = reward_function.get_statistics()
print("\n📊 Reward/Parsing stats after training:")
for k, v in stats.items():
    print(f"  {k}: {v:.4f}" if isinstance(v, float) else f"  {k}: {v}")

print("\n✅ Training complete!")

🚀 Starting RL training...
This may take a while. Monitor progress in TensorBoard.

To view TensorBoard: tensorboard --logdir outputs/tau_bench_rl


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 926 | Num Epochs = 2 | Total steps = 462
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 16
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 16 x 1) = 32
 "-____-"     Trainable parameters = 7,962,624 of 20,922,719,808 (0.04% trained)
[unsloth_zoo.log|INFO]Unsloth: num_items_in_batch = None
`generation_config` default values have been modified to match model-specific defaults: {'max_length': 131072}. If this is not desired, please set these values explicitly.
Unsloth: Compiling kernels: 100%|███████████| 9/9 [00:00<00:00, 48.16it/s, triton_per_fused__to_copy_mul_stack_sum_transpose_unsqueeze_view_8]
[unsloth_compiled_module_gpt_oss|INFO]Unsloth: Removing compiler guards after 1 inference run. DYNAMO_STANCE.stance = eager_on_recompile DYNAMO_STANCE.skip_guard_eval_unsafe = False
Unsloth: Compiling kernels: 100%|█████████████████████| 1/1 [00:00<00:00, 40

Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,kl,rewards / reward_function_wrapper_with_metrics / mean,rewards / reward_function_wrapper_with_metrics / std
10,0.034700,4.378125,0.484657,79.728125,58.000000,132.800000,0.000000,79.728125,58.000000,132.800000,34.652643,4.378125,1.051976
20,0.016900,4.426875,0.484207,80.093750,56.800000,121.900000,0.000000,80.093750,56.800000,121.900000,16.916614,4.426875,1.072855
30,0.001400,4.239062,0.577798,84.256250,48.600000,147.300000,0.000000,84.256250,48.600000,147.300000,1.440271,4.239062,1.350824
40,0.001300,4.535313,0.599672,88.550000,58.100000,221.500000,0.006250,85.937802,58.100000,166.600000,1.301024,4.535313,1.482719
50,0.016200,4.679375,0.512087,84.756250,55.500000,179.700000,0.003125,83.424194,55.500000,141.400000,16.240725,4.679375,1.262728
60,0.002100,4.540625,0.448929,82.034375,61.600000,142.700000,0.000000,82.034375,61.600000,142.700000,2.068457,4.540625,1.097267
70,0.001600,4.693125,0.497791,88.146875,64.300000,147.700000,0.000000,88.146875,64.300000,147.700000,1.603288,4.693125,1.097443
80,0.004200,4.428437,0.308690,83.259375,66.200000,115.100000,0.000000,83.259375,66.200000,115.100000,4.164466,4.428437,1.226090
90,0.004900,4.952812,0.377855,90.228125,67.000000,141.300000,0.000000,90.228125,67.000000,141.300000,4.858206,4.952812,1.300032
100,3.395400,5.110625,0.271687,88.303125,73.600000,138.700000,0.000000,88.303125,73.600000,138.700000,3395.370618,5.110625,1.207892


[unsloth_zoo.log|INFO]Unsloth: num_items_in_batch = None
[unsloth_compiled_module_gpt_oss|INFO]Unsloth: Reseting guards. DYNAMO_STANCE.stance = default DYNAMO_STANCE.skip_guard_eval_unsafe = False
Unsloth: Compiling kernels: 100%|███████████████████████████| 2/2 [00:00<00:00, 35.53it/s, triton_poi_fused_add_cat_mul_split_sub_unsqueeze_1]
[unsloth_compiled_module_gpt_oss|INFO]Unsloth: Removing compiler guards after 1 inference run. DYNAMO_STANCE.stance = eager_on_recompile DYNAMO_STANCE.skip_guard_eval_unsafe = False
[unsloth_zoo.log|INFO]Unsloth: num_items_in_batch = None
[unsloth_compiled_module_gpt_oss|INFO]Unsloth: Reseting guards. DYNAMO_STANCE.stance = default DYNAMO_STANCE.skip_guard_eval_unsafe = False
[unsloth_compiled_module_gpt_oss|INFO]Unsloth: Removing compiler guards after 1 inference run. DYNAMO_STANCE.stance = eager_on_recompile DYNAMO_STANCE.skip_guard_eval_unsafe = False
[unsloth_zoo.log|INFO]Unsloth: num_items_in_batch = None
[unsloth_compiled_module_gpt_oss|INFO]Unsl


🔍 Running validation at step 100


...


📊 Validation Results:
  ├─ Average Reward:  4.830
  ├─ Success Rate:    100.0%
  ├─ Parse Failures:  0/10
  └─ Time:            238.5s



[unsloth_compiled_module_gpt_oss|INFO]Unsloth: Reseting guards. DYNAMO_STANCE.stance = default DYNAMO_STANCE.skip_guard_eval_unsafe = False
[unsloth_compiled_module_gpt_oss|INFO]Unsloth: Removing compiler guards after 1 inference run. DYNAMO_STANCE.stance = eager_on_recompile DYNAMO_STANCE.skip_guard_eval_unsafe = False
[unsloth_zoo.log|INFO]Unsloth: num_items_in_batch = None
[unsloth_compiled_module_gpt_oss|INFO]Unsloth: Reseting guards. DYNAMO_STANCE.stance = default DYNAMO_STANCE.skip_guard_eval_unsafe = False
[unsloth_compiled_module_gpt_oss|INFO]Unsloth: Removing compiler guards after 1 inference run. DYNAMO_STANCE.stance = eager_on_recompile DYNAMO_STANCE.skip_guard_eval_unsafe = False
[unsloth_zoo.log|INFO]Unsloth: num_items_in_batch = None
[unsloth_compiled_module_gpt_oss|INFO]Unsloth: Reseting guards. DYNAMO_STANCE.stance = default DYNAMO_STANCE.skip_guard_eval_unsafe = False
[unsloth_compiled_module_gpt_oss|INFO]Unsloth: Removing compiler guards after 1 inference run. DYNAMO_


🔍 Running validation at step 462


[unsloth_compiled_module_gpt_oss|INFO]Unsloth: Reseting guards. DYNAMO_STANCE.stance = default DYNAMO_STANCE.skip_guard_eval_unsafe = False
[unsloth_compiled_module_gpt_oss|INFO]Unsloth: Removing compiler guards after 1 inference run. DYNAMO_STANCE.stance = eager_on_recompile DYNAMO_STANCE.skip_guard_eval_unsafe = False
[unsloth_compiled_module_gpt_oss|INFO]Unsloth: Reseting guards. DYNAMO_STANCE.stance = default DYNAMO_STANCE.skip_guard_eval_unsafe = False
[unsloth_compiled_module_gpt_oss|INFO]Unsloth: Removing compiler guards after 1 inference run. DYNAMO_STANCE.stance = eager_on_recompile DYNAMO_STANCE.skip_guard_eval_unsafe = False
[unsloth_compiled_module_gpt_oss|INFO]Unsloth: Reseting guards. DYNAMO_STANCE.stance = default DYNAMO_STANCE.skip_guard_eval_unsafe = False
[unsloth_compiled_module_gpt_oss|INFO]Unsloth: Removing compiler guards after 1 inference run. DYNAMO_STANCE.stance = eager_on_recompile DYNAMO_STANCE.skip_guard_eval_unsafe = False
[unsloth_compiled_module_gpt_oss|I


📊 Validation Results:
  ├─ Average Reward:  6.020
  ├─ Success Rate:    100.0%
  ├─ Parse Failures:  0/10
  └─ Time:            218.6s


📊 Reward/Parsing stats after training:
  task_count: 14812
  average_reward: 4.8627
  zero_reward_count: 20
  zero_reward_rate: 0.0014
  parse_failures: 1
  parse_failure_rate: 0.0001

✅ Training complete!


In [31]:
## Print Comprehensive Training Metrics Report

print("\n" + "="*100)
print("COMPREHENSIVE TRAINING METRICS ANALYSIS")
print("="*100)

# Print the comprehensive report
metrics_tracker.print_training_report()

# Print detailed per-epoch breakdown
print("\n" + "="*100)
print("DETAILED EPOCH-BY-EPOCH BREAKDOWN")
print("="*100)

for epoch in sorted(metrics_tracker.epoch_rewards.keys()):
    metrics_tracker.print_epoch_report(epoch)

# Save metrics to JSON for later analysis
import json

metrics_dict = {
    'epoch_summaries': [
        metrics_tracker.get_epoch_summary(epoch)
        for epoch in sorted(metrics_tracker.epoch_rewards.keys())
    ],
    'step_summaries': [
        metrics_tracker.get_step_summary(step)
        for step in sorted(metrics_tracker.step_rewards.keys())
        if step % 50 == 0  # Sample every 50 steps to avoid too much data
    ]
}

metrics_file = f"{output_dir}/training_metrics.json"
import os
os.makedirs(os.path.dirname(metrics_file), exist_ok=True)
with open(metrics_file, 'w') as f:
    json.dump(metrics_dict, f, indent=2)

print(f"\n✅ Detailed metrics saved to: {metrics_file}")
print("\n📊 Key Takeaways:")
print("  ✓ Reward trajectory shows training progress")
print("  ✓ Parse failure rate indicates response quality")
print("  ✓ Zero-reward rate shows penalty effectiveness")
print("  ✓ Action correctness (tools vs args) guides model improvement")


COMPREHENSIVE TRAINING METRICS ANALYSIS

COMPREHENSIVE TRAINING METRICS REPORT

📈 Epoch Summaries:

📊 Epoch 0 Summary:
  Examples: 14784
  Reward Trajectory:
    ├─ Mean: 4.8534 ± 1.4928
    ├─ Range: [-0.5000, 10.0000]
    ├─ Median: 5.0000
    └─ Positive Rate: 99.70%

📊 Overall Training Statistics:
  Total Examples Trained: 14784
  Reward Mean: 4.8534
  Reward Range: [-0.5000, 10.0000]
  Positive Rewards: 14740 (99.70%)
  Zero/Negative Rewards: 44 (0.30%)

DETAILED EPOCH-BY-EPOCH BREAKDOWN

📊 Epoch 0 Summary:
  Examples: 14784
  Reward Trajectory:
    ├─ Mean: 4.8534 ± 1.4928
    ├─ Range: [-0.5000, 10.0000]
    ├─ Median: 5.0000
    └─ Positive Rate: 99.70%

✅ Detailed metrics saved to: outputs/tau_bench_rl/training_metrics.json

📊 Key Takeaways:
  ✓ Reward trajectory shows training progress
  ✓ Parse failure rate indicates response quality
  ✓ Zero-reward rate shows penalty effectiveness
  ✓ Action correctness (tools vs args) guides model improvement


## Validation History

Review validation metrics collected during training.

In [32]:
# Display validation history from training
if validation_callback.validation_history:
    print("\n📊 Validation History During Training:")
    print("=" * 80)
    print(f"{'Step':<10} {'Avg Reward':<15} {'Success Rate':<15} {'Parse Failures':<15}")
    print("=" * 80)
    
    for entry in validation_callback.validation_history:
        print(f"{entry['step']:<10} {entry['avg_reward']:<15.3f} {entry['success_rate']:<15.1%} {entry['parse_failures']:<15}")
    
    print("=" * 80)
    
    # Plot validation curve if multiple checkpoints
    if len(validation_callback.validation_history) > 1:
        print("\n📈 Validation Trend:")
        steps = [e['step'] for e in validation_callback.validation_history]
        rewards = [e['avg_reward'] for e in validation_callback.validation_history]
        
        best_idx = rewards.index(max(rewards))
        best_step = steps[best_idx]
        best_reward = rewards[best_idx]
        
        print(f"  ├─ Best validation reward: {best_reward:.3f} at step {best_step}")
        print(f"  ├─ Final validation reward: {rewards[-1]:.3f}")
        print(f"  └─ Improvement: {rewards[-1] - rewards[0]:.3f}")
else:
    print("\n⚠️  No validation history recorded (training may have been skipped)")


📊 Validation History During Training:
Step       Avg Reward      Success Rate    Parse Failures 
100        4.740           100.0%          0              
200        5.020           100.0%          0              
300        4.470           100.0%          0              
400        4.830           100.0%          0              
462        6.020           100.0%          0              

📈 Validation Trend:
  ├─ Best validation reward: 6.020 at step 462
  ├─ Final validation reward: 6.020
  └─ Improvement: 1.280


## Save Trained Model

Save the trained LoRA adapters and tokenizer.

In [33]:
save_path = f"{output_dir}/final_model"

model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"✓ Model saved to {save_path}")

✓ Model saved to outputs/tau_bench_rl/final_model


## Final Evaluation on Test Set

Evaluate the trained model on the held-out TEST split for unbiased performance metrics.

Test the trained model on sample tasks.

In [34]:
def evaluate_model(model, tokenizer, rl_tasks: List[RLTask], reward_function, num_samples: int = 10, task_id_to_chatml: dict = None):
    """Evaluate the trained model using ChatML prompts (consistent with training).
    
    Args:
        task_id_to_chatml: Optional dict mapping task_id to ChatML prompts. 
                          If None, uses TASK_ID_TO_CHATML (for train tasks).
                          For test evaluation, pass TEST_TASK_ID_TO_CHATML.
    """
    eval_tasks = random.sample(rl_tasks, min(num_samples, len(rl_tasks)))
    
    # Use provided mapping or default to training task mapping
    chatml_map = task_id_to_chatml if task_id_to_chatml is not None else TASK_ID_TO_CHATML
    
    total_reward = 0.0
    results = []
    
    for task in tqdm(eval_tasks, desc="Evaluating"):
        # Use ChatML prompt for generation (same as training)
        chatml_prompt = chatml_map.get(task.task_id)
        if not chatml_prompt:
            # Fallback: regenerate ChatML prompt on the fly
            user_query = task.prompt.split("Query:", 1)[-1].strip() if "Query:" in task.prompt else task.prompt
            messages = [
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": user_query},
            ]
            chatml_prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        
        inputs = tokenizer(chatml_prompt, return_tensors="pt").to(model.device)
        input_length = inputs['input_ids'].shape[1]
        
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=256,
                temperature=0.2,  # keep deterministic behavior
                top_p=0.9,
                do_sample=False,
                pad_token_id=tokenizer.eos_token_id,
            )
        
        # Extract only the generated tokens (skip the input tokens)
        generated_tokens = outputs[0][input_length:]
        response = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()
        
        # Skip if response is empty
        if not response:
            logger.warning(f"Empty response generated for task {task.task_id}")
            results.append({
                'task_id': task.task_id,
                'reward': 0.0,
                'metrics': {'error': 'Empty response'},
                'response': ''
            })
            continue
        
        # Normalize + strict validate before scoring
        norm = normalize_response_to_json(response, task)
        if not strict_validate_json_response(norm):
            reward, metrics = -0.5, {'error': 'invalid_json'}  # soft penalty consistent with training
        else:
            reward, metrics = reward_function.calculate_reward(norm, task)
        total_reward += reward
        
        results.append({
            'task_id': task.task_id,
            'reward': reward,
            'metrics': metrics,
            'response': (norm if len(norm) < 500 else norm[:500])
        })
    
    avg_reward = total_reward / len(eval_tasks) if eval_tasks else 0.0
    return avg_reward, results

# Evaluate
print("\n📊 Evaluating trained model on TEST set...")
print("⚠️  Using held-out TEST split for final unbiased evaluation")

# Set model to eval mode
model.eval()

# IMPORTANT: Use TEST split for final evaluation (not val_tasks!)
avg_reward, eval_results = evaluate_model(
    model, 
    tokenizer, 
    test_tasks,  # Use test split for final evaluation
    reward_function, 
    num_samples=len(test_tasks),  # Evaluate on ALL test tasks for comprehensive results
)

# Set model back to train mode if needed
model.train()

print(f"\n✅ Evaluation complete!")
print(f"   Test Set Size: {len(test_tasks)} tasks")
print(f"   Average Reward: {avg_reward:.3f}")
print(f"   Success Rate: {sum(1 for r in eval_results if r['reward'] > 0) / len(eval_results):.1%}")
print(f"\nTop 3 results:")
for i, result in enumerate(eval_results[:3], 1):
    print(f"\n{i}. Task {result['task_id']}: ")
    print(f"   Reward: {result['reward']:.3f}")
    print(f"   Response: {result['response'][:100]}...")
print(f"\n✅ Evaluation complete!")


📊 Evaluating trained model on TEST set...
⚠️  Using held-out TEST split for final unbiased evaluation


Evaluating:   0%|                                                                                                     | 0/199 [00:00<?, ?it/s][unsloth_compiled_module_gpt_oss|INFO]Unsloth: Reseting guards. DYNAMO_STANCE.stance = default DYNAMO_STANCE.skip_guard_eval_unsafe = False
[unsloth_compiled_module_gpt_oss|INFO]Unsloth: Removing compiler guards after 1 inference run. DYNAMO_STANCE.stance = eager_on_recompile DYNAMO_STANCE.skip_guard_eval_unsafe = False
Evaluating:   1%|▍                                                                                          | 1/199 [00:19<1:05:13, 19.77s/it][unsloth_compiled_module_gpt_oss|INFO]Unsloth: Reseting guards. DYNAMO_STANCE.stance = default DYNAMO_STANCE.skip_guard_eval_unsafe = False
[unsloth_compiled_module_gpt_oss|INFO]Unsloth: Removing compiler guards after 1 inference run. DYNAMO_STANCE.stance = eager_on_recompile DYNAMO_STANCE.skip_guard_eval_unsafe = False
Evaluating:   1%|▉                                                      


✅ Evaluation complete!
   Test Set Size: 199 tasks
   Average Reward: 5.062
   Success Rate: 100.0%

Top 3 results:

1. Task task_1508: 
   Reward: 5.000
   Response: {"tool_calls":[{"name":"find_user_id_by_email","arguments":{"email":"sofia.muller5339@example.com"}}...

2. Task task_620: 
   Reward: 8.000
   Response: {"tool_calls":[{"name":"find_user_id_by_email","arguments":{"email":"mia.jackson2679@example.com"}},...

3. Task task_386: 
   Reward: 8.000
   Response: {"tool_calls":[{"name":"find_user_id_by_email","arguments":{"email":"yusuf.khan7390@example.com"}},{...

✅ Evaluation complete!


In [35]:
# DIAGNOSTIC CELL: Detailed evaluation analysis
print("\n DETAILED EVALUATION DIAGNOSTICS")
print("=" * 60)

# Test with a single task first
model.eval()
test_task = rl_tasks[0]

# ✅ FIX: Use ChatML format (same as training)
chatml_prompt = TASK_ID_TO_CHATML.get(test_task.task_id)
if not chatml_prompt:
    # Fallback: regenerate ChatML prompt
    user_query = test_task.prompt.split("Query:", 1)[-1].strip() if "Query:" in test_task.prompt else test_task.prompt
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_query},
    ]
    chatml_prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

inputs = tokenizer(chatml_prompt, return_tensors="pt").to(model.device)
input_length = inputs['input_ids'].shape[1]

print(f"\nTask: {test_task.task_id}")
print(f"Prompt length: {input_length} tokens")
print(f"Expected actions: {test_task.gold_actions_str[:150]}")
print(f"\nGenerating response...")

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        temperature=0.3,  # Very low temp for testing
        do_sample=False,  # Greedy decoding for reproducibility
        pad_token_id=tokenizer.eos_token_id,
    )

generated_tokens = outputs[0][input_length:]
response = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()

print(f"\nGenerated response ({len(response)} chars):")
print(f"---START---")
print(response[:500])
print(f"---END---")

# Try to parse
print(f"\nParsing response...")
try:
    parsed = reward_function._parse_response_to_actions(response)
    print(f"✓ Parsed {len(parsed)} actions")
    for p in parsed:
        print(f"  - {p.name}: {p.arguments}")
except Exception as e:
    print(f"✗ Parse error: {e}")

# Calculate reward
try:
    reward, metrics = reward_function.calculate_reward(response, test_task)
    print(f"\nReward calculation:")
    print(f"  Total reward: {reward:.3f}")
    print(f"  Metrics: {json.dumps(metrics, indent=4)}")
except Exception as e:
    print(f"✗ Reward error: {e}")
    import traceback
    traceback.print_exc()

model.train()



 DETAILED EVALUATION DIAGNOSTICS

Task: task_1
Prompt length: 516 tokens
Expected actions: [{"name": "find_user_id_by_email", "arguments": {"email": "omar.santos1752@example.com"}}, {"name": "get_order_details", "arguments": {"order_id": "#W

Generating response...


[unsloth_compiled_module_gpt_oss|INFO]Unsloth: Reseting guards. DYNAMO_STANCE.stance = default DYNAMO_STANCE.skip_guard_eval_unsafe = False
[unsloth_compiled_module_gpt_oss|INFO]Unsloth: Removing compiler guards after 1 inference run. DYNAMO_STANCE.stance = eager_on_recompile DYNAMO_STANCE.skip_guard_eval_unsafe = False



Generated response (295 chars):
---START---
{"tool_calls":[{"name":"find_user_id_by_email","arguments":{"email":"omar.santos1752@example.com"}},{"name":"get_order_details","arguments":{"order_id":"#W9121070"}},{"name":"cancel_pending_order","arguments":{"order_id":"#W9121070","reason":"Customer requested cancellation before shipment"}}]}
---END---

Parsing response...
✓ Parsed 3 actions
  - find_user_id_by_email: {'email': 'omar.santos1752@example.com'}
  - get_order_details: {'order_id': '#W9121070'}
  - cancel_pending_order: {'order_id': '#W9121070', 'reason': 'Customer requested cancellation before shipment'}

Reward calculation:
  Total reward: 4.500
  Metrics: {
    "correct_action_names": 3,
    "correct_arguments": 2,
    "incorrect_actions": 0,
    "correct_ordering": true,
    "total_expected": 3,
    "total_parsed": 3,
    "precision": 1.0,
    "recall": 1.0,
    "f1": 1.0,
    "reward": 4.5,
    "completeness_penalty": 0.0,
    "missing_args_details": [],
    "parse_failure

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GptOssForCausalLM(
      (model): GptOssModel(
        (embed_tokens): Embedding(201088, 2880, padding_idx=199999)
        (layers): ModuleList(
          (0-23): 24 x GptOssDecoderLayer(
            (self_attn): GptOssAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2880, out_features=4096, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2880, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): 

## Save Evaluation Results

In [36]:
eval_path = f"{output_dir}/evaluation_results.json"

with open(eval_path, 'w', encoding='utf-8') as f:
    json.dump({
        'average_reward': avg_reward,
        'num_samples': len(eval_results),
        'results': eval_results
    }, f, indent=2)

print(f"✓ Evaluation results saved to {eval_path}")

✓ Evaluation results saved to outputs/tau_bench_rl/evaluation_results.json


## Export for Inference

Optionally merge LoRA weights for faster inference.

In [37]:
# Merge LoRA weights into base model for faster inference
model.save_pretrained_merged(
    f"{output_dir}/merged_model",
    tokenizer,
    save_method="merged_16bit",  # Use "merged_4bit" for quantized
)

print(f"✓ Merged model saved to {output_dir}/merged_model")
print("\n🎉 All done! Your model is ready for deployment.")

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub
Checking cache directory for required files...
Cache check failed: model-00000-of-00002.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Preparing safetensor model files: 100%|██████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 7252.40it/s]


Note: tokenizer.model not found (this is OK for non-SentencePiece models)


Unsloth: Merging weights into 16bit: 100%|██████████████████████████████████████████████████████████████████████| 3/3 [04:15<00:00, 85.31s/it]


Unsloth: Regenerating safetensors index for dequantized MXFP4 model...
Unsloth: Merge process complete. Saved to `/home/asus/rl_grpo/outputs/tau_bench_rl/merged_model`
✓ Merged model saved to outputs/tau_bench_rl/merged_model

🎉 All done! Your model is ready for deployment.


## Summary

**What we did:**
1. ✅ Loaded GPT-OSS 20B with Unsloth optimizations
2. ✅ Applied LoRA for efficient RL training
3. ✅ Loaded tau-bench tasks from dataset.jsonl
4. ✅ Defined reward function based on action correctness
5. ✅ Trained using GRPO (Group Relative Policy Optimization)
6. ✅ Evaluated and saved the trained model